In [1]:
import requests

In [2]:
from openai import OpenAI

In [3]:
import json

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv()

True

In [6]:
import os

In [87]:
stock_api = os.getenv("ALPHA_VANTAGE_API_KEY")

In [187]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [29]:
from agents import Agent, Runner, function_tool, ModelSettings, SQLiteSession

In [10]:
from functools import lru_cache

In [126]:
from datetime import datetime, timedelta

In [11]:
url = "https://www.alphavantage.co/query?"

In [12]:
import json

In [13]:
@function_tool
def find_ticker(text: str) -> str:
    """Based on the natural language query, use this function to find the ticker symbol"""

    # params = {
    #     "function": "SYMBOL_SEARCH",
    #     "keywords": text,
    #     "apikey": stock_api
    # }

    # response = requests.get(url, params)

    # data = response.json()

    # ticker = (data.get('bestMatches')[0]).get('1. symbol')

    return _find_ticker(text)

In [14]:
@lru_cache(maxsize = 100)
def _find_ticker(text: str) -> str:
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": text,
        "apikey": stock_api
    }

    response = requests.get(url, params)

    data = response.json()

    ticker = data.get('bestMatches')

    filter_region = []

    for t in ticker:
        if t.get('4. region') == 'United States':
            filter_region.append(t)

    return filter_region[0].get('1. symbol')

In [16]:
_find_ticker('Microsoft')

'MSFT'

In [15]:
@function_tool
def get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """
    # params = {
    #     "function": "INCOME_STATEMENT",
    #     "symbol": ticker,
    #     "apikey": stock_api
    # }

    # income_statement = requests.get(url,params)

    # income_statement_data = income_statement.json()

    # annual_data = income_statement_data.get('annualReports')

    # filtered_data = []

    # for report in annual_data:
    #     filtered_data.append({
    #         'fiscalDateEnding': report.get('fiscalDateEnding'),
    #         'grossProfit': report.get('grossProfit'),
    #         'totalRevenue': report.get('totalRevenue'),
    #         'costOfRevenue': report.get('costOfRevenue'),
    #         'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
    #         'operatingIncome': report.get('operatingIncome'),
    #         'operatingExpenses': report.get('operatingExpenses'),
    #         'researchAndDevelopment': report.get('researchAndDevelopment'),
    #         'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
    #         'netInterestIncome': report.get('netInterestIncome'),
    #         'interestIncome': report.get('interestIncome'),
    #         'depreciationAndAmortization': report.get('depreciationAndAmortization'),
    #         'incomeBeforeTax': report.get('incomeBeforeTax'),
    #         'incomeTaxExpense': report.get('incomeTaxExpense'),
    #         'ebit': report.get('ebit'),
    #         'ebitda': report.get('ebitda'),
    #         'netIncome': report.get('netIncome'),
    #     })

    return _get_income_statement(ticker)

In [16]:
@lru_cache(maxsize = 100)
def _get_income_statement(ticker: str) -> str:
    """Based on the ticker you can find the income statement of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": stock_api
    }

    income_statement = requests.get(url,params)

    income_statement_data = income_statement.json()

    annual_data = income_statement_data.get('annualReports')

    filtered_data = []

    for report in annual_data:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    return filtered_data

In [19]:
_get_income_statement.cache_info()

CacheInfo(hits=0, misses=0, maxsize=100, currsize=0)

In [115]:
# find_ticker('tesla')

In [17]:
@function_tool
def get_annual_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement.
    """

    annual_filter = json_file.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })

    return filtered_data

In [101]:
get_annual_income_statement(cached_income_statement)

[{'fiscalDateEnding': '2024-12-31',
  'grossProfit': '17450000000',
  'totalRevenue': '97690000000',
  'costOfRevenue': '80240000000',
  'costofGoodsAndServicesSold': '80240000000',
  'operatingIncome': '10374000000',
  'researchAndDevelopment': '4540000000',
  'sellingGeneralAndAdministrative': '5150000000',
  'netInterestIncome': '1219000000',
  'interestIncome': '1569000000',
  'depreciationAndAmortization': '5368000000',
  'incomeBeforeTax': '8990000000',
  'incomeTaxExpense': '1837000000',
  'ebit': '9340000000',
  'ebitda': '14708000000',
  'netIncome': '7130000000'},
 {'fiscalDateEnding': '2023-12-31',
  'grossProfit': '17660000000',
  'totalRevenue': '96773000000',
  'costOfRevenue': '79113000000',
  'costofGoodsAndServicesSold': '79113000000',
  'operatingIncome': '8769000000',
  'researchAndDevelopment': '3969000000',
  'sellingGeneralAndAdministrative': '4800000000',
  'netInterestIncome': '910000000',
  'interestIncome': '1066000000',
  'depreciationAndAmortization': '46670

In [18]:
@function_tool
def get_quarterly_income_statement(json_file):

    """ 
    You are supposed to use this function, if the query is related to finding annual income statement datapoints.
    """

    quarterly_filter = json_file.get('quarterlyReports')

    filtered_qoq_data = []

    for report in quarterly_filter:
        filtered_qoq_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'grossProfit': report.get('grossProfit'),
            'totalRevenue': report.get('totalRevenue'),
            'costOfRevenue': report.get('costOfRevenue'),
            'costofGoodsAndServicesSold': report.get('costofGoodsAndServicesSold'),
            'operatingIncome': report.get('operatingIncome'),
            'operatingExpenses': report.get('operatingExpenses'),
            'researchAndDevelopment': report.get('researchAndDevelopment'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'netInterestIncome': report.get('netInterestIncome'),
            'interestIncome': report.get('interestIncome'),
            'depreciationAndAmortization': report.get('depreciationAndAmortization'),
            'incomeBeforeTax': report.get('incomeBeforeTax'),
            'incomeTaxExpense': report.get('incomeTaxExpense'),
            'ebit': report.get('ebit'),
            'ebitda': report.get('ebitda'),
            'netIncome': report.get('netIncome'),
        })


    return filtered_qoq_data

In [109]:
get_quarterly_income_statement(cached_income_statement)

[{'fiscalDateEnding': '2025-09-30',
  'grossProfit': '5054000000',
  'totalRevenue': '28095000000',
  'costOfRevenue': '23041000000',
  'costofGoodsAndServicesSold': '23041000000',
  'operatingIncome': '3430000000',
  'researchAndDevelopment': '1630000000',
  'sellingGeneralAndAdministrative': '1562000000',
  'netInterestIncome': '363000000',
  'interestIncome': '439000000',
  'depreciationAndAmortization': '1625000000',
  'incomeBeforeTax': '1959000000',
  'incomeTaxExpense': '570000000',
  'ebit': '2035000000',
  'ebitda': '3660000000',
  'netIncome': '1373000000'},
 {'fiscalDateEnding': '2025-06-30',
  'grossProfit': '3878000000',
  'totalRevenue': '22496000000',
  'costOfRevenue': '18618000000',
  'costofGoodsAndServicesSold': '18618000000',
  'operatingIncome': '2955000000',
  'researchAndDevelopment': '1589000000',
  'sellingGeneralAndAdministrative': '1366000000',
  'netInterestIncome': '306000000',
  'interestIncome': '392000000',
  'depreciationAndAmortization': '1433000000',


In [19]:
@function_tool
def get_balance_sheet(ticker: str) -> str:
    """Based on the ticker you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual income statements.
    """

    # params = {
    #     "function": "BALANCE_SHEET",
    #     "symbol": ticker,
    #     "apikey": stock_api
    # }

    # balance_sheet = requests.get(url,params)

    # balance_sheet_data = balance_sheet.json()

    # annual_data = balance_sheet_data.get('annualReports')

    # filtered_data = []

    # for report in annual_data:
    #     filtered_data.append({
    #         'fiscalDateEnding': report.get('fiscalDateEnding'),
    #         'totalAssets': report.get('totalAssets'),
    #         'totalCurrentAssets': report.get('totalCurrentAssets'),
    #         'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
    #         'inventory': report.get('inventory'),
    #         'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
    #         'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
    #         'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
    #         'totalLiabilities': report.get('totalLiabilities'),
    #         'interestIncome': report.get('interestIncome'),
    #         'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
    #         'currentLongTermDebt': report.get('currentLongTermDebt'),
    #         'totalShareholderEquity': report.get('totalShareholderEquity'),
    #         'retainedEarnings': report.get('retainedEarnings'),
    #     })

    return _get_balance_sheet(ticker)

In [20]:
@lru_cache
def _get_balance_sheet(ticker: str) -> str:

    params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": stock_api
    }

    balance_sheet = requests.get(url,params)

    balance_sheet_data = balance_sheet.json()

    annual_data = balance_sheet_data.get('annualReports')

    filtered_data = []

    for report in annual_data:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
        })

    return filtered_data

In [128]:
get_balance_sheet('TSLA')

[{'fiscalDateEnding': '2024-12-31',
  'totalAssets': '122070000000',
  'totalCurrentAssets': '58360000000',
  'cashAndCashEquivalentsAtCarryingValue': '16139000000',
  'inventory': '12017000000',
  'totalNonCurrentAssets': '63716000000',
  'accumulatedDepreciationAmortizationPPE': 'None',
  'sellingGeneralAndAdministrative': None,
  'totalLiabilities': '48390000000',
  'interestIncome': None,
  'totalCurrentLiabilities': '28821000000',
  'currentLongTermDebt': '2343000000',
  'totalShareholderEquity': '72913000000',
  'retainedEarnings': '35209000000'},
 {'fiscalDateEnding': '2023-12-31',
  'totalAssets': '106618000000',
  'totalCurrentAssets': '49616000000',
  'cashAndCashEquivalentsAtCarryingValue': '16398000000',
  'inventory': '13626000000',
  'totalNonCurrentAssets': '57003000000',
  'accumulatedDepreciationAmortizationPPE': 'None',
  'sellingGeneralAndAdministrative': None,
  'totalLiabilities': '43009000000',
  'interestIncome': None,
  'totalCurrentLiabilities': '28748000000',


In [119]:
cached_balance_sheet = get_balance_sheet(ticker_symbol)

cached_balance_sheet

{'symbol': 'TSLA',
 'annualReports': [{'fiscalDateEnding': '2024-12-31',
   'reportedCurrency': 'USD',
   'totalAssets': '122070000000',
   'totalCurrentAssets': '58360000000',
   'cashAndCashEquivalentsAtCarryingValue': '16139000000',
   'cashAndShortTermInvestments': '16139000000',
   'inventory': '12017000000',
   'currentNetReceivables': '4418000000',
   'totalNonCurrentAssets': '63716000000',
   'propertyPlantEquipment': '51501000000',
   'accumulatedDepreciationAmortizationPPE': 'None',
   'intangibleAssets': '1226000000',
   'intangibleAssetsExcludingGoodwill': '1226000000',
   'goodwill': '244000000',
   'investments': 'None',
   'longTermInvestments': 'None',
   'shortTermInvestments': '20424000000',
   'otherCurrentAssets': '5362000000',
   'otherNonCurrentAssets': 'None',
   'totalLiabilities': '48390000000',
   'totalCurrentLiabilities': '28821000000',
   'currentAccountsPayable': '12474000000',
   'deferredRevenue': 'None',
   'currentDebt': 'None',
   'shortTermDebt': '32

In [21]:
@function_tool
def annual_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding annual balance sheet datapoints.
    """

    annual_filter = json_data.get('annualReports')

    filtered_data = []

    for report in annual_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [22]:
@function_tool
def quarterly_balance_sheet(json_data):

    """ 
    You are supposed to use this function, if the query is related to finding quarterly balance sheet datapoints.
    """

    quarterly_filter = json_data.get('quarterlyReports')

    filtered_data = []

    for report in quarterly_filter:
        filtered_data.append({
            'fiscalDateEnding': report.get('fiscalDateEnding'),
            'totalAssets': report.get('totalAssets'),
            'totalCurrentAssets': report.get('totalCurrentAssets'),
            'cashAndCashEquivalentsAtCarryingValue': report.get('cashAndCashEquivalentsAtCarryingValue'),
            'inventory': report.get('inventory'),
            'totalNonCurrentAssets': report.get('totalNonCurrentAssets'),
            'accumulatedDepreciationAmortizationPPE': report.get('accumulatedDepreciationAmortizationPPE'),
            'sellingGeneralAndAdministrative': report.get('sellingGeneralAndAdministrative'),
            'totalLiabilities': report.get('totalLiabilities'),
            'interestIncome': report.get('interestIncome'),
            'totalCurrentLiabilities': report.get('totalCurrentLiabilities'),
            'currentLongTermDebt': report.get('currentLongTermDebt'),
            'totalShareholderEquity': report.get('totalShareholderEquity'),
            'retainedEarnings': report.get('retainedEarnings'),
            'commonStock': report.get('commonStock')
        })

    return filtered_data

In [23]:
@function_tool
def cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    # params = {
    #     "function": "CASH_FLOW",
    #     "symbol": ticker,
    #     "apikey": stock_api
    # }

    # cash_flow = requests.get(url,params)

    # cash_flow_data = cash_flow.json()

    # annual_filter = cash_flow_data.get('annualReports')

    # final_cash_flow_data = []

    # for cash_flow in annual_filter:
    #     final_cash_flow_data.append({
    #         'fiscalDateEnding': cash_flow.get('fiscalDateEnding'),
    #         'operatingCashflow': cash_flow.get('operatingCashflow'),
    #         'cashflowFromInvestment': cash_flow.get('cashflowFromInvestment'),
    #         'cashflowFromFinancing': cash_flow.get('cashflowFromFinancing'),
    #         'netIncome': cash_flow.get('netIncome')
    #     })

    return _cash_flow_statement(ticker)

In [24]:
@lru_cache
def _cash_flow_statement(ticker: str) -> str:
    """Based on the ticker found out using the function find_ticker(text) you can find the balance sheet of the company from this
    function. This will fetch both quarterly and annual cash flow statement.
    """

    params = {
        "function": "CASH_FLOW",
        "symbol": ticker,
        "apikey": stock_api
    }

    cash_flow = requests.get(url,params)

    cash_flow_data = cash_flow.json()

    annual_filter = cash_flow_data.get('annualReports')

    final_cash_flow_data = []

    for cash_flow in annual_filter:
        final_cash_flow_data.append({
            'fiscalDateEnding': cash_flow.get('fiscalDateEnding'),
            'operatingCashflow': cash_flow.get('operatingCashflow'),
            'cashflowFromInvestment': cash_flow.get('cashflowFromInvestment'),
            'cashflowFromFinancing': cash_flow.get('cashflowFromFinancing'),
            'netIncome': cash_flow.get('netIncome')
        })

    return final_cash_flow_data

In [134]:
cash_flow_statement(ticker_symbol)

[{'fiscalDateEnding': '2024-12-31',
  'operatingCashflow': '14923000000',
  'cashflowFromInvestment': '-18787000000',
  'cashflowFromFinancing': '3853000000',
  'netIncome': '7153000000'},
 {'fiscalDateEnding': '2023-12-31',
  'operatingCashflow': '13256000000',
  'cashflowFromInvestment': '-15584000000',
  'cashflowFromFinancing': '2589000000',
  'netIncome': '14974000000'},
 {'fiscalDateEnding': '2022-12-31',
  'operatingCashflow': '14724000000',
  'cashflowFromInvestment': '-11973000000',
  'cashflowFromFinancing': '-3527000000',
  'netIncome': '12556000000'},
 {'fiscalDateEnding': '2021-12-31',
  'operatingCashflow': '11497000000',
  'cashflowFromInvestment': '-7868000000',
  'cashflowFromFinancing': '-5203000000',
  'netIncome': '5519000000'},
 {'fiscalDateEnding': '2020-12-31',
  'operatingCashflow': '5943000000',
  'cashflowFromInvestment': '-3132000000',
  'cashflowFromFinancing': '9973000000',
  'netIncome': '690000000'},
 {'fiscalDateEnding': '2019-12-31',
  'operatingCashflo

In [25]:
@function_tool
def price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    # params = {
    #     "function": "TIME_SERIES_DAILY",
    #     "symbol": ticker,
    #     "outputsize": "compact",
    #     "apikey": stock_api
    # }

    # response = requests.get(url,params)

    # response_json = response.json()

    # filtered_json = response_json['Time Series (Daily)']

    # close_price = []

    # for date, values in filtered_json.items():
    #     close_price.append(
    #         {
    #             'date': date,
    #             'close': values['4. close']
    #         }
    #     )

    # latest_price = close_price[0]

    return _price_data(ticker)

In [26]:
@lru_cache
def _price_data(ticker: str) -> str:
    """ 
    Based on the ticker, we need to find the price of the stock through this.
    """

    params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": ticker,
        "outputsize": "compact",
        "apikey": stock_api
    }

    response = requests.get(url,params)

    response_json = response.json()

    filtered_json = response_json['Time Series (Daily)']

    close_price = []

    for date, values in filtered_json.items():
        close_price.append(
            {
                'date': date,
                'close': values['4. close']
            }
        )

    latest_price = close_price[0]

    return latest_price

In [173]:
@function_tool
def earnings_analysis(ticker: str) -> str:
    """ 
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    return _earnings_analysis(ticker)

In [174]:
@lru_cache
def _earnings_analysis(ticker: str) -> str:
    """
    Based on the ticker, you will find a 10-12 point analysis related to the latest earnings report. 
    You can use it in case the user wants to do some kind of qualitative analysis
    """

    final_transcript = []

    final_quarter = []

    while final_transcript == []:
        today = datetime.now()
    
        year = today.year
        month = today.month
    
        if month in [1,2,3]:
            quarter = 'Q4'
            year -= 1
        elif month in [4,5,6]:
            quarter = 'Q1'
        elif month in [7,8,9]:
            quarter = 'Q2'
        else:
            quarter = 'Q3'
    
        final = f'{year}{quarter}'
    
        params = {
            "function": "EARNINGS_CALL_TRANSCRIPT",
            "symbol": ticker,
            "apikey": stock_api,
            "quarter": final
        }

        response = requests.get(url, params)

        final_json = response.json()

        transcript = final_json.get('transcript')

        quarter = final_json.get('quarter')

        final_transcript.append(transcript)

        final_quarter.append(quarter)

    
    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": f""" 
           You will be given the full earnings call transcript in {final_transcript}.

            Your task is to extract **7–10 investor-relevant insights** strictly based on what 
            **Apple management said** in the transcript.
            
            Rules you MUST follow:
            1. Use ONLY information explicitly stated by Apple executives (CEO, CFO, or Investor Relations).  
               Do NOT include analyst opinions, assumptions, or external interpretation.
            2. Each point must represent a **distinct insight**, not a restatement of another point.
            3. For every point, clearly mention **who said it** (name + role).
            4. Do NOT use generic phrases such as “strong performance”, “positive outlook”, or “solid results” 
            unless backed by specific data or commentary.
            5. Each point must explain **why this matters to a long-term investor** 
            (growth, margins, cash flow, competitive position, or strategic direction).
            6. Prefer **synthesis over repetition** — combine numbers with management commentary where relevant.
            7. Do NOT introduce any facts, estimates, or forward views not explicitly stated in the transcript.
            
            Output format:
            - Heading: Analysis of the earnings call ({final_quarter})
            - Numbered list (7–10 points)
            - 2–4 sentences per point
            - End each point with: *(Source: Name, Title)*
            
            Tone:
            - Analytical
            - Investor-oriented
            - Neutral and factual

            Don't only include positive or negative points, try to include both
            """}
        ], temperature = 0.3
    )

    return response.choices[0].message.content

In [172]:
_earnings_analysis('MSFT')

"# Analysis of the earnings call (['2025Q3'])\n\n1. Microsoft Cloud revenue surpassed $42 billion, reflecting a 22% growth in constant currency. This significant increase underscores the company's strong position in the cloud market, which is critical for long-term growth as businesses increasingly migrate to cloud solutions. The robust demand for cloud services indicates that Microsoft is effectively capturing market share and expanding its customer base. *(Source: Satya Nadella, CEO)*\n\n2. The company reported a commercial bookings increase of 18%, driven by Azure commitments from major clients like OpenAI. This growth in bookings is a positive indicator of future revenue, as it suggests that customers are making long-term commitments to Microsoft's cloud services, which can enhance revenue predictability and stability. *(Source: Amy Hood, CFO)*\n\n3. Microsoft’s commercial remaining performance obligation rose to $315 billion, up 34% year-over-year. This metric is crucial as it ref

In [27]:
async def agent_builder(query):

    agent = Agent(
        name = "Data Finder",
        instructions = """Your job is to understand the user query and use the tools, to answer the user query. 
        Finally take all the relavant data and convert it into a JSON in the following format:
    
        Return ONLY valid JSON in the following format with all the numbers.
    
        {
          "ticker": "TSLA",
          "stock_price": {
            "date": "YYYY-MM-DD",
            "close": number
          },
          "income_statement": {
            "fiscalDateEnding": "YYYY-MM-DD",
            "totalRevenue": number,
            "grossProfit": number,
            "operatingIncome": number,
            "netIncome": number
            <All the other details that you are getting with the income statement>
          },
          "balance_sheet": {
            "fiscalDateEnding": "YYYY-MM-DD",
            "totalAssets": number,
            "totalLiabilities": number,
            "totalShareholderEquity": number,
            "cashAndCashEquivalents": number
            <All the other details that you are getting with the balance sheet>
          },
          "cash_flow": {
            "fiscalDateEnding": "YYYY-MM-DD",
            "operatingCashFlow": number,
            "cashFlowFromInvesting": number,
            "cashFlowFromFinancing": number
            <All the other details that you are getting with the cash flow statement>
          }
        }
        
        Do not include explanations or additional text.
    
        Important Rule:
        Based on the type of query, call the <tools>. If there is a tool which is not needed to be called, don't call it and just put that 
        specific data as null.
        
        
        """,
        tools = [
            price_data,
            cash_flow_statement,
            find_ticker,
            get_balance_sheet,
            get_income_statement,
        ],
        model_settings = ModelSettings(tool_choice = "auto")
    )

    result = await Runner.run(agent, query)

    final_result = result.final_output

    convert_json = json.loads(final_result)

    response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": f""" 
            You are a professional stock analyst providing clear, data-driven insights.

            User asked: "{query}"
            
            Instructions:
            1. Answer the user's specific question directly
            2. Use concrete numbers from {convert_json} to support your points
            3. Keep response conversational but professional
            4. If asked for investment advice, provide analysis but remind them you're not a financial advisor
            5. Structure: Brief intro → Key points with data → Conclusion
            6. Length: 150-250 words
            
            Provide your analysis.
            """}
        ], temperature = 0.3
    )

    return response.choices[0].message.content

In [49]:
await agent_builder("Give me details about the tesla stock?")

"Tesla, Inc. (TSLA) has been a focal point in the stock market, and as of December 24, 2025, its stock price closed at $485.40. Let’s delve into some key financial metrics that provide insight into the company's performance.\n\nFor the fiscal year ending December 31, 2024, Tesla reported total revenues of approximately $97.69 billion, with a gross profit of $17.45 billion. The operating income stood at $7.08 billion, leading to a net income of $7.13 billion after accounting for an income tax expense of $1.84 billion. This indicates a healthy profitability margin, showcasing Tesla's ability to convert revenue into profit effectively.\n\nOn the balance sheet, Tesla's total assets were valued at $122.07 billion, against total liabilities of $48.39 billion, resulting in total shareholder equity of $72.91 billion. The company's cash and cash equivalents amounted to $16.14 billion, providing a solid liquidity position.\n\nFrom a cash flow perspective, Tesla generated $14.92 billion in operat

In [52]:
await agent_builder("Compare the performance of Oracle and Salesforce, and choose which one is a better bet?")

"When comparing Oracle (ORCL) and Salesforce (CRM), both companies exhibit strong fundamentals, but their performance metrics suggest different strengths.\n\n**Oracle** has a stock price of $197.49 as of December 24, 2025. For the fiscal year ending May 31, 2025, Oracle reported total revenue of $57.4 billion and a net income of $12.4 billion. The company has a solid operating income of $17.7 billion and a gross profit of $40.5 billion. Its balance sheet shows total assets of $168.4 billion against total liabilities of $147.4 billion, indicating a manageable debt level with total shareholder equity of $20.5 billion.\n\n**Salesforce**, on the other hand, closed at $265.26 on the same date. For the fiscal year ending January 31, 2025, Salesforce generated total revenue of $37.9 billion and a net income of $6.2 billion. The operating income stood at $7.2 billion, with a gross profit of $29.3 billion. Salesforce's balance sheet reflects total assets of $102.9 billion and total liabilities 

In [71]:
global_session = SQLiteSession("test_conversation")

In [183]:
async def _agent_builder(query):

    agent = Agent(
        name = "Data Finder",
        # model = 'gpt-4o-mini',
        instructions = f"""
        You are a stock analyser and your job is to understand the {query} and then use the tools available at your disposal 
        to answer to the user. 

        Make sure that you ONLY get data from tools and in case the query cannot be solved by tools, empathetically refuse to answer
        the user query. 

        It is recommended you use tables, if you're doing any comparison either between 2 or more companies or you're 
        showing the data of more than 1 year. 

        You can use the {earnings_analysis} tool in case the user wants to know what happened in the latest earnings call of a particular
        company.

        However, you should always try to back it up with financial data first. 
        
        EXCEPTION:
        If the user is specifically asking to summarise the earning call data you are supposed to simply just print out
        the output you're getting from {earnings_analysis}.
        """,
        tools = [
            price_data,
            cash_flow_statement,
            find_ticker,
            get_balance_sheet,
            get_income_statement,
            earnings_analysis
        ],
        model_settings = ModelSettings(
            tool_choice = "auto",
            seed = 0,
            temperature = 0.3
        )
    )
    
    result = await Runner.run(agent, query, session = global_session)

    return print(result.final_output)

In [184]:
#testing earning analysis

await _agent_builder("What happened in the latest earnings call of microsoft?")

Here’s a summary of what happened in the latest Microsoft earnings call (2025 Q3):

1. Microsoft Cloud surpassed $42 billion in revenue, growing 22% in constant currency.
2. Commercial bookings increased by 18%, driven by Azure commitments from OpenAI.
3. Microsoft’s AI business revenue exceeded expectations, with Azure AI services contributing significantly to growth.
4. The gross margin percentage for Microsoft Cloud was 69%, down three points year-over-year due to scaling AI infrastructure.
5. Microsoft 365 commercial cloud revenue grew 12%, driven by increased adoption of E5 and M365 Copilot.
6. Capital expenditures for the quarter were $21.4 billion, slightly lower than expected, showing efficiency in data center build-outs.
7. Commercial remaining performance obligation reached $315 billion, up 34% year-over-year.
8. Paid Microsoft 365 commercial seats grew 7% year-over-year to over 430 million.
9. AI services led to a 16-point growth rate lift to Azure.
10. Microsoft is continuo

In [181]:
await _agent_builder("Give an overall review of Google, are they on the right track?")

Here’s an overall review of Google (Alphabet Inc.) based on the latest financial and qualitative data:

### Financial Performance (2024 vs. Previous Years)

| Year  | Total Revenue | Net Income   | Operating Income | Free Cash Flow (TTM) |
|-------|--------------|--------------|------------------|----------------------|
| 2024  | $350.0B      | $100.1B      | $112.4B          | $73.6B               |
| 2023  | $307.4B      | $73.8B       | $84.3B           | —                    |
| 2022  | $282.8B      | $60.0B       | $74.8B           | —                    |

- **Revenue and profit are at all-time highs,** with 2024 marking Alphabet’s first-ever $100 billion quarter.
- **Operating margins** remain strong (30.5%), indicating efficient cost management.

### Strategic & Operational Highlights

- **AI Leadership:** Alphabet’s Gemini AI app has over 650 million monthly active users, and AI is deeply integrated into products like Search and YouTube.
- **Cloud Growth:** Google Cloud revenu

In [79]:
#testing session settings
await _agent_builder("I want income details related to Alphabet Inc?")

Here are the key income details for Alphabet Inc. (GOOGL) for the past 5 years:

| Fiscal Year | Total Revenue | Gross Profit | Operating Income | Net Income   | EBITDA        |
|-------------|--------------|--------------|------------------|--------------|--------------|
| 2024        | $350.02B     | $203.71B     | $112.39B         | $100.12B     | $135.39B     |
| 2023        | $307.39B     | $174.06B     | $84.29B          | $73.80B      | $97.97B      |
| 2022        | $282.84B     | $156.63B     | $74.84B          | $59.97B      | $85.16B      |
| 2021        | $257.64B     | $146.70B     | $78.71B          | $76.03B      | $103.52B     |
| 2020        | $182.53B     | $97.80B      | $41.22B          | $40.27B      | $61.91B      |

**Key Terms:**
- **Total Revenue:** All income from sales and services.
- **Gross Profit:** Revenue minus cost of goods/services sold.
- **Operating Income:** Profit from core business operations.
- **Net Income:** Profit after all expenses, taxes, an

In [80]:
#testing session settings
await _agent_builder("Can you give me a detailed breakdown of R&D expenses?")

Here is a detailed breakdown of Alphabet Inc.'s (GOOGL) Research & Development (R&D) expenses for the past 5 years:

| Fiscal Year | R&D Expenses (USD) |
|-------------|--------------------|
| 2024        | $49,326,000,000    |
| 2023        | $45,427,000,000    |
| 2022        | $39,500,000,000    |
| 2021        | $31,562,000,000    |
| 2020        | $27,573,000,000    |

**Observations:**
- Alphabet’s R&D spending has consistently increased year-over-year.
- In 2024, R&D expenses reached nearly $50 billion, highlighting Alphabet’s strong focus on innovation and technology development.

If you want a longer historical view or a comparison with other companies, just let me know!


In [68]:
#testing session settings
await _agent_builder("Do you think it is justified to spend so much on R&D?")

I understand your interest in whether Alphabet Inc.'s high R&D spending is justified. However, as an AI, I can only provide factual data and cannot offer personal opinions or subjective analysis. 

If you would like, I can provide you with:
- R&D as a percentage of total revenue over the years
- Comparison of Alphabet’s R&D spending with other major tech companies

Let me know how you’d like to proceed!


In [69]:
#testing session settings
await _agent_builder("Why are they spending so much on R&D? Is it their largest expense?")

Let's analyze Alphabet Inc.'s (GOOGL) expenses to answer your questions:

### 1. Why is Alphabet spending so much on R&D?
Alphabet is a technology-driven company whose core businesses (Google Search, YouTube, Android, Cloud, AI, etc.) rely heavily on innovation. High R&D spending allows Alphabet to:
- Maintain leadership in search, advertising, and cloud services
- Develop new products and services (e.g., AI, self-driving cars, quantum computing)
- Stay competitive against other tech giants

### 2. Is R&D their largest expense?
Let's compare R&D expenses with other major expense categories for the last 3 years:

| Fiscal Year | R&D Expense | Cost of Revenue | SG&A Expense | Operating Expenses (Total) |
|-------------|-------------|-----------------|--------------|---------------------------|
| 2024        | $49.33B     | $146.31B        | $14.19B      | $91.32B                   |
| 2023        | $45.43B     | $133.33B        | $16.43B      | $89.77B                   |
| 2022        |

In [54]:
await agent_builder("Give me details about the tesla stock?")

"Tesla, Inc. (ticker: TSLA) has been a prominent player in the electric vehicle market, and as of December 24, 2025, its stock price closed at $485.40. Let’s delve into some key financial metrics to better understand its current standing.\n\nFor the fiscal year ending December 31, 2024, Tesla reported total revenue of approximately $97.69 billion, showcasing robust growth. The company achieved a gross profit of $17.45 billion, indicating a healthy gross margin. However, the cost of revenue was significant at $80.24 billion, which highlights the challenges of scaling production and maintaining profitability.\n\nOperating income stood at $7.08 billion, with operating expenses totaling $10.37 billion, reflecting substantial investments in research and development ($4.54 billion) and selling, general, and administrative costs ($5.15 billion). The net income for the year was $7.13 billion, after accounting for an income tax expense of $1.84 billion.\n\nOn the balance sheet, Tesla had total 

In [93]:
await _agent_builder("Give me details about the tesla stock?")

Here are the latest details about Tesla (TSLA) stock, including its price, key financials, and recent performance:

---

### Tesla (TSLA) Stock Snapshot

| Metric                | Value (Latest)      |
|-----------------------|---------------------|
| Stock Price (as of 2025-12-24) | $485.40            |
| Fiscal Year End       | December 31, 2024   |

---

### Key Financials (Annual)

#### Income Statement

| Year | Total Revenue | Net Income | Gross Profit | Operating Income | EBITDA      |
|------|--------------|------------|--------------|------------------|-------------|
| 2024 | $97.69B      | $7.13B     | $17.45B      | $7.08B           | $14.71B     |
| 2023 | $96.77B      | $14.97B    | $17.66B      | $8.89B           | $14.80B     |
| 2022 | $81.46B      | $12.59B    | $20.85B      | $13.66B          | $17.83B     |

#### Balance Sheet

| Year | Total Assets | Total Liabilities | Shareholder Equity | Cash & Equivalents |
|------|-------------|-------------------|-------------

In [95]:
#gpt 4.1
await _agent_builder("Help me understand the financial performance of Alphabet Inc, is it a good buy?")

Here’s a comprehensive analysis of Alphabet Inc.’s (GOOGL) financial performance based on the latest available data:

### 1. Income Statement (Key Metrics, USD Millions)

| Year      | Revenue     | Net Income | Gross Profit | Operating Income | EPS (if available) |
|-----------|-------------|------------|--------------|------------------|--------------------|
| 2024      | 350,018     | 100,118    | 203,712      | 112,390          | N/A                |
| 2023      | 307,394     | 73,795     | 174,062      | 84,293           | N/A                |
| 2022      | 282,836     | 59,972     | 156,633      | 74,842           | N/A                |
| 2021      | 257,637     | 76,033     | 146,698      | 78,714           | N/A                |
| 2020      | 182,527     | 40,269     | 97,795       | 41,224           | N/A                |

**Observations:**
- Revenue and net income have shown strong growth, especially in the last two years.
- Operating income and gross profit margins are robus

In [96]:
#gpt-4.1
await _agent_builder("Compare the performance of Oracle and Salesforce, and choose which one is a better bet?")

Here is a comprehensive comparison of Oracle (ORCL) and Salesforce (CRM) based on their latest financial performance, balance sheets, and cash flows. This should help determine which company is a better bet as of the most recent fiscal years.

---

### Stock Price (as of 2025-12-24)
| Company    | Ticker | Price (USD) |
|------------|--------|-------------|
| Oracle     | ORCL   | 197.49      |
| Salesforce | CRM    | 265.26      |

---

### Income Statement (Most Recent Fiscal Year)
| Metric                | Oracle (2025)      | Salesforce (2025)  |
|-----------------------|--------------------|--------------------|
| Total Revenue         | $57.40B            | $37.90B            |
| Gross Profit          | $40.47B            | $29.25B            |
| Operating Income      | $17.68B            | $7.21B             |
| Net Income            | $12.44B            | $6.20B             |
| EBITDA                | $23.91B            | $11.14B            |

---

### Balance Sheet (Most Recen

In [97]:
#gpt 4.1
await _agent_builder("Help me understand the revenue of Alphabet Inc, is it growing year over year?")

Here is a summary of Alphabet Inc.'s (Google's) annual revenue for the past several years:

| Year | Total Revenue (USD) |
|------|---------------------|
| 2024 | $350,018,000,000    |
| 2023 | $307,394,000,000    |
| 2022 | $282,836,000,000    |
| 2021 | $257,637,000,000    |
| 2020 | $182,527,000,000    |
| 2019 | $161,857,000,000    |
| 2018 | $136,819,000,000    |
| 2017 | $110,855,000,000    |
| 2016 | $90,272,000,000     |
| 2015 | $74,989,000,000     |

**Analysis:**
- Alphabet Inc.'s revenue has consistently grown year over year.
- From 2020 to 2024, revenue increased from $182.5 billion to $350.0 billion.
- The growth trend is strong, with each year showing a significant increase over the previous year.

**Conclusion:**  
Yes, Alphabet Inc.'s revenue is growing year over year, and the growth rate has been robust, especially in recent years. If you need a more detailed analysis or growth rate calculation, let me know!


In [98]:
#gpt-4.1
await _agent_builder("Compare the performance of Oracle and Salesforce, and choose which one is a better bet?")

Here’s a detailed comparison of Oracle (ORCL) and Salesforce (CRM) based on their latest financials, stock price, and key performance metrics. All data is from their most recent annual filings.

---

### 1. Stock Price (as of Dec 24, 2025)

| Company    | Stock Price (USD) |
|------------|-------------------|
| Oracle     | $197.49           |
| Salesforce | $265.26           |

---

### 2. Income Statement (FY 2025)

| Metric              | Oracle (ORCL)      | Salesforce (CRM)   |
|---------------------|--------------------|--------------------|
| Total Revenue       | $57.40B            | $37.90B            |
| Gross Profit        | $40.47B            | $29.25B            |
| Operating Income    | $17.68B            | $7.21B             |
| Net Income          | $12.44B            | $6.20B             |
| EBITDA              | $23.91B            | $11.14B            |
| Net Margin          | 21.7%              | 16.3%              |

---

### 3. Balance Sheet (FY 2025)

| Metric    

In [99]:
#gpt 4.1
await _agent_builder("Help me understand the financial performance of Alphabet Inc, is it a good buy?")

Here’s a comprehensive analysis of Alphabet Inc. (GOOGL) based on its latest financial statements and stock price. This will help you understand its financial performance and whether it might be a good buy.

---

### 1. Income Statement Highlights (Annual)

| Year       | Revenue ($B) | Net Income ($B) | Gross Profit ($B) | Operating Income ($B) |
|------------|--------------|-----------------|-------------------|-----------------------|
| 2024*      | 350.0        | 100.1           | 203.7             | 112.4                 |
| 2023       | 307.4        | 73.8            | 174.1             | 84.3                  |
| 2022       | 282.8        | 59.9            | 156.6             | 74.8                  |
| 2021       | 257.6        | 76.0            | 146.7             | 78.7                  |
| 2020       | 182.5        | 40.3            | 97.8              | 41.2                  |

*2024 numbers are the latest available.

**Key Takeaways:**
- Revenue and net income have grown s

In [142]:
a.get('transcript')

[{'speaker': 'Operator',
  'title': 'Operator',
  'content': "Good day, and welcome to the Apple Q1 Fiscal Year 2024 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions. I would like to turn the call over to Suhasini Chandramouli, Director of Investor Relations. Please go ahead.",
  'sentiment': '0.0'},
 {'speaker': 'Suhasini Chandramouli',
  'title': 'Director of Investor Relations',
  'content': "Thank you for joining us. Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Before turning the call over to Tim, I would like to remind everyone that the quarter we're reporting today included 13 weeks, whereas the quarter we reported a year ago included 14 weeks. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding re

In [89]:
a = {'symbol': 'AAPL', 'quarter': '2024Q1', 'transcript': [{'speaker': 'Operator', 'title': 'Operator', 'content': "Good day, and welcome to the Apple Q1 Fiscal Year 2024 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions. I would like to turn the call over to Suhasini Chandramouli, Director of Investor Relations. Please go ahead.", 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': "Thank you for joining us. Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Before turning the call over to Tim, I would like to remind everyone that the quarter we're reporting today included 13 weeks, whereas the quarter we reported a year ago included 14 weeks. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation and future business outlook, including the potential impact of macroeconomic conditions on the company's business and results of operations. These statements involve risks and uncertainties that may cause actual results or trends to differ materially from our forecast. For more information, please refer to the risk factors discussed in Apple's most recently filed annual report on Form 10-K and the form 8-K filed with the SEC today along with the associated press release. Apple assumes no obligation to update any forward-looking statements, which speak only as of the date they are made. I'd now like to turn the call over to Tim for introductory remarks.", 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Thank you, Suhasini. Good afternoon, everyone, and thanks for joining the call. Today, Apple is reporting revenue of $119.6 billion for the December quarter, up 2% from a year ago despite having one less week in the quarter. EPS was $2.18, up 16% from a year ago and an all-time record. We achieved revenue records across more than two dozen countries and regions including all-time records in Europe and the rest of Asia-Pacific. We also continue to see strong double-digit growth in many emerging markets with all-time records in Malaysia, Mexico, The Philippines, Poland, and Turkey, as well as December quarter records in India, Indonesia, Saudi Arabia, and Chile. In Services, we set an all-time revenue record with paid subscriptions growing double digits year-over-year. And I'm pleased to announce today that we have set a new record for our installed base, which has now surpassed 2.2 billion active devices. We are announcing these results on the eve of what is sure to be a historic day as we enter the era of spatial computing. Starting tomorrow, Apple Vision Pro, the most advanced personal electronics device ever, will be available in Apple stores for customers in the U.S. with expansion to other countries later this year. Apple Vision Pro is a revolutionary device built on decades of Apple innovation and it's years ahead of anything else. Apple Vision Pro has a groundbreaking new input system and thousands of innovations, and it will unlock incredible experiences for users and developers that are simply not possible on any other device. There is already so much excitement behind this product from reviewers, customers, and developers. They are praising everything from the incredible experience of watching a movie on a 100-foot screen to remarkable new machine learning capabilities like hand tracking and room mapping. We can't wait for people to experience the magic for themselves. Moments like these are what we live for at Apple. They're why we do what we do. They're why we're so unflinchingly dedicated to groundbreaking innovation and why we're so focused on pushing technology to its limits as we work to enrich the lives of our users. As we look ahead, we will continue to invest in these and other technologies that will shape the future. That includes artificial intelligence, where we continue to spend a tremendous amount of time and effort, and we're excited to share the details of our ongoing work in that space later this year. Now, let's turn to the results for the December quarter, beginning with iPhone. We are proud to report that revenue came in at $69.7 billion, 6% higher than a year ago. The iPhone 15 lineup has earned glowing reviews and been embraced by customers. The iPhone 15 and iPhone 15 Plus feature a gorgeous new design with color-infused back glass and contoured edges, Dynamic Island, A16 Bionic, and a new 48 megapixel camera system. And the iPhone 15 Pro and iPhone 15 Pro Max set the gold standard for smartphones with a beautiful and lighter titanium design, industry-leading performance with A17 Pro, and our most advanced camera system with the equivalent of seven pro lenses and the ability to record spatial video. Features like Emergency SOS and roadside assistance via satellite bring peace of mind to users when they travel, and I'm grateful for every note I've received about their lifesaving impact. Turning to Mac. Revenue came in at $7.8 billion, up 1% year-over-year, driven by the strength of our latest M3-powered MacBook Pro models in spite of having one less week of sales. Just last week, we got to wish Mac a happy 40th birthday. When it was introduced 40 years ago, Mac changed everything, and through the years, it has done so again and again. Recently, we have been on a tremendous pace of innovation. Since the introduction of Apple silicon in 2020, we've been proud to offer our users unmatched performance and power along with a remarkable Neural Engine for artificial intelligence and machine learning. This past fall, we had an amazing launch of the latest generation of Apple silicon for Mac, M3, M3 Pro, and M3 Max. These chips break new ground in power and performance empowering users to do more than they ever could before, whether they're making a musical masterpiece using the latest features in Logic Pro, or beating their high score in a graphics intensive game. A favorite amongst students, business owners, artists, and video editors, our MacBook Pro lineup is the world's best pro notebook family. And iMac, the world's most capable and best-selling all-in-one, is now faster than ever, thanks to M3. In iPad, revenue for the December quarter was $7 billion, down 25% year-over-year due to a difficult comparison with the launch of the M2 iPad Pro and the 10th generation iPad during the December quarter last year and one less week of sales. iPad remains the most versatile, capable, and elegant tablet on the market today. It continues to be the go-to device for students, creators, and more with customers loving iPad's incredible combination of portability and performance. Powerful apps like Final Cut Pro and Logic Pro for iPad allow video and music creators to unleash their creativity in new ways that are only possible on iPad. iPad continues to push the boundaries of what's possible on a tablet. In Wearables, Home, and Accessories, revenue came in at $12 billion, down 11% from a year ago due to a difficult comparison with the launch timing of several products in this category and the impact of the 14th week last year. Across our latest Apple Watch lineup, we're enabling and encouraging our users to live a healthier day, while making Apple Watch even more intuitive to use. The new double-tap gesture on Apple Watch Series 9 and Apple Watch Ultra 2 makes it easier to answer calls, play and pause music, or take a photo with iPhone. I've been deeply moved by the many touching stories about how features like a regular rhythm notification and fall detection helped Apple Watch users when they needed it most. And for the first time ever, users can choose a carbon-neutral option of any new Apple Watch. Meanwhile, our AirPods lineup continues to be a holiday favorite. In Services, we set an all-time revenue record of $23.1 billion, an 11% year-over-year increase. Because we had one less week this quarter, this growth represents an acceleration from the September quarter, and we achieved all-time revenue records across advertising, cloud services, payment services, and video, as well as December quarter records in the App Store and AppleCare. Across our services, we're constantly growing our offerings to give users even more to love. With the redesigned Apple TV app, we've made it easier for subscribers to enjoy all their favorite shows, movies, and sports, including Apple TV+ hits like Masters of the Air, Monarch, and Slow Horses. We're proud to be a part of Martin Scorsese's Killers of the Flower Moon, a film that has moved audiences and earned more than 200 accolades including Best Film of the Year from the New York Film Critics Circle, nine BAFTA nominations, a Golden Globe win, and 10 Oscar nominations, including Best Picture. Across all Apple TV+ productions, we've now earned 2050 award nominations and 450 wins since we've introduced the service. We're also excited to have a new season of Major League Soccer kicking off this month. We're looking forward to seeing Lionel Messi return to the field and to following all of our favorite teams in what is sure to be an incredible season. And we're counting down to the Apple Music Super Bowl halftime show, featuring Usher. Turning to Retail. In recent months, we opened three stores, including our 100th store in Asia-Pacific. Throughout the holidays, our team members pulled out all the stops to help customers find the perfect gift. And I know our U.S. team members are especially excited to begin demoing Apple Vision Pro for our customers tomorrow. At Apple, we live and breathe innovation. We are driven to pioneer new technology that can enrich our customers' lives, and we're just as intentional about showing up with our values and being a force for good in the world. February is Black History Month, and to honor it, we've launched our new Black Unity Collection, which includes the Black Unity Sport Loop band. This year's designs reflect a lasting commitment to working toward a more equitable world. We also continue to do key work through our Racial Equity and Justice Initiative, and we're proud to continue providing grants to organizations that are making a real impact in the world. In recent months, we've also taken significant strides in our environmental work. We're partnering with suppliers to bring more clean energy online for Apple production. We're using more recycled materials than ever before and more energy-efficient transportation than ever before. And each day, we are taking more and more steps toward becoming 100% carbon-neutral across all of our products by 2030. Apple is a company that has never shied away from big challenges. That's because we are grounded by a deep sense of purpose and guided by a core belief in the transformative power of innovation. And so, we are optimistic about the future, confident in the long term, and as excited as we've ever been to deliver for our users like only Apple can. With that, I'll turn it over to Luca.", 'sentiment': '0.7'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Thank you, Tim, and good afternoon, everyone. Revenue for the December quarter was $119.6 billion, up 2% from last year. During the December quarter a year ago, two unique factors affected our results. First, we had an additional week in the quarter. And second, we had COVID-related factory shutdowns that limited iPhone supply. We estimate that the net impact of these two factors resulted in a 2 percentage point headwind to our revenue performance this quarter. We set all-time revenue records in Europe and the rest of Asia-Pacific and continue to see strong performance across our emerging markets with double-digit growth in the majority of the emerging markets we track. Products revenue was $96.5 billion, flat compared to last year, driven by strength in iPhone, offset by challenging comparisons for iPad and Wearables, Home, and Accessories and one less week of sales this year across the entire portfolio. Thanks to our unparalleled customer loyalty and very strong levels of customer satisfaction, our total installed base of active devices set a new record across all products and all geographic segments, and is now over 2.2 billion active devices. Services revenue set an all-time record of $23.1 billion, up 11% year-over-year. When we take into account the extra week last year, this represents a sequential acceleration of growth from the September quarter. We are very pleased with our Services performance in both developed and emerging markets with all-time revenue records in the Americas, Europe, and the rest of Asia-Pacific. Company gross margin was 45.9%, up 70 basis points sequentially, driven by leverage and favorable mix, partially offset by foreign exchange. Products gross margin was 39.4%, up 280 basis points sequentially, also driven by leverage and mix, partially offset by foreign exchange. Services gross margin was 72.8%, up 190 basis points from last quarter, due to a more favorable mix. Operating expenses of $14.5 billion were at the midpoint of the guidance range we provided and up 1% year-over-year. Net income was $33.9 billion, up $3.9 billion from last year. Diluted EPS was $2.18, up 16% versus last year and an all-time record. And operating cash flow was very strong at $39.9 billion. Let me now provide more detail for each of our revenue categories. iPhone revenue was $69.7 billion, up 6% year-over-year. We set all-time records in several countries and regions, including Latin America, Western Europe, the Middle East, and Korea, as well as December quarter records in India and Indonesia. Our iPhone active installed base grew to a new all-time high, and we had an all-time record number of iPhone upgraders during the quarter. Customers are loving their new iPhone 15 family, with the latest reports from 451 Research indicating customer satisfaction of 99% in the U.S. In fact, many iPhone models were among the top-selling smartphones around the world during the quarter. According to a survey from Kantar, iPhones were four out of the top five models in the U.S. and Japan, four out of the top six models in urban China and the UK, and all top five models in Australia. Mac generated revenue of $7.8 billion and returned to growth, despite one less week of sales this year. This represents a significant acceleration from the September quarter when we faced a challenging compare due to the supply disruptions and subsequent demand recapture we experienced a year ago. Customer response to our latest iMac and MacBook Pro models powered by the M3 chips has been great. And our Mac installed base reached an all-time high with almost half of Mac buyers during the quarter being new to the product. Also, 451 Research recently reported customer satisfaction of 97% for Mac in the U.S. iPad generated $7 billion in revenue, down 25% year-over-year. iPad faced a difficult compare because during the December quarter last year, we launched the new iPad Pro and iPad 10 generation, and we had an extra week of sales. However, the iPad installed base continues to grow and is at an all-time high with over half of the customers who purchased iPads during the quarter being new to the product, and customer satisfaction for iPad was recently measured at 98% in the U.S. Wearables, Home, and Accessories revenue was $12 billion, down 11% year-over-year due to a challenging launch comparison and the extra week a year ago. This time last year, we had the full quarter benefit from the launches of the AirPods Pro 2nd generation, the Watch SE, and the first Watch Ultra. We continue to attract new customers to Apple Watch. Nearly two-thirds of customers purchasing an Apple Watch during the quarter were new to the product, and the latest reports from 451 Research indicate customer satisfaction of 96% in the U.S. And in Services, we were very pleased with our double-digit growth, which was driven by the strength of our ecosystem. Our installed base is now over 2.2 billion active devices and continues to grow nicely, establishing a solid foundation for the future expansion of our Services business. And we continue to see increased customer engagement with our services. Both transacting accounts and paid accounts reached a new all-time high, with paid accounts growing double digits year-over-year. Also, our paid subscriptions showed strong double-digit growth. We have well over 1 billion paid subscriptions across the services on our platform, more than double the number that we had only four years ago. Finally, we continue to build on the breadth and quality of our current services. From Oscar-nominated theatrical releases with Apple TV+ to more publications or News+ like The Atlantic and exciting new games on Arcade. Turning to Enterprise, we continue to see many business customers leverage Apple products to improve productivity and drive innovation. Target recently added the latest M3 MacBook Pro to their existing deployment of thousands of Macs, enabling employees across various departments to do their best work. In emerging markets, Zoho, a leading technology company headquartered in India, offers its 15,000-plus global employees a choice of devices, with 80% of their workforce using iPhone for work and nearly two-thirds of them choosing Mac as their primary computer. With the upcoming launch of Apple Vision Pro, we are seeing strong excitement in Enterprise. Leading organizations across many industries such as Walmart, Nike, Vanguard, Stryker, Bloomberg, and SAP have started leveraging and investing in Apple Vision Pro as their new platform to bring innovative spatial computing experiences to their customers and employees. From everyday productivity to collaborative product design to immersive training, we cannot wait to see the amazing things our enterprise customers will create in the months and years to come. Let me now turn to our cash position and capital return program. We ended the quarter with $173 billion in cash and marketable securities. We decreased commercial paper by $4 billion, leaving us with total debt of $108 billion. As a result, net cash was $65 billion at the end of the quarter, and our goal of becoming net cash-neutral over time remains unchanged. During the quarter, we returned nearly $27 billion to shareholders, including $3.8 billion in dividends and equivalents and $20.5 billion through open market repurchases of 112 million Apple shares. We also retired an additional 6 million shares in the final settlement of our 19th ASR. As usual, we will provide an update to our capital return program when we report results at the end of this quarter. As we move ahead into the March quarter, I'd like to review our outlook, which includes the types of forward-looking information that Suhasini referred to at the beginning of the call. The color we are providing today assumes that the macroeconomic outlook doesn't worsen from what we are projecting today for the current quarter. And we expect foreign exchange to be a revenue headwind of about 2 percentage points on a year-over-year basis. As a reminder, in the December quarter a year ago, we faced significant supply constraints on the iPhone 14 Pro and 14 Pro Max due to COVID-19 factory shutdowns. And in the March quarter a year ago, we were able to replenish channel inventory and fulfill significant pent-up demand from the constraints. We estimate that this impact added close to $5 billion to the March quarter's total revenue last year. When we remove this impact from last year's revenue, we expect both our March quarter total company revenue and iPhone revenue to be similar to a year ago. For our Services business, we expect a similar double-digit growth rate to what we reported in the December quarter. We expect gross margin to be between 46% and 47%. We expect OpEx to be between $14.3 billion and $14.5 billion. We expect OI&E to be around $50 million, excluding any potential impact from the mark-to-market of minority investments and our tax rate to be around 16%. Finally, today our Board of Directors has declared a cash dividend of $0.24 per share of common stock payable on February 15, 2024, to shareholders of record as of February 12, 2024. With that, let's open the call to questions.", 'sentiment': '0.5'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Luca. We ask that you limit yourself to two questions. Operator, may we have the first question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': "Certainly. We'll go ahead and take our first question from Erik Woodring with Morgan Stanley. Please go ahead.", 'sentiment': '0.0'}, {'speaker': 'Erik Woodring', 'title': 'Analyst', 'content': 'Hey guys, good evening. Thank you for taking my questions. I have two. And congrats on the nice quarter here. Luca, maybe if we start with you. Can you unpackage some of the Services drivers a bit for us? Obviously, really nice outperformance in the December quarter versus your expectations. A record gross margin implies your higher margin businesses were likely the sources of outperformance. But can you maybe just clarify a bit how we should think about Services growth for the March quarter? And then, speak to some of those underlying drivers in the December quarter and then in the March quarter, what the different puts and takes would be? And then, I have a follow-up please. Thank you.', 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Thank you for the question. Let's begin with the December quarter. As mentioned, we saw an 11% increase, reaching $23.1 billion, which is a record for us, with significant achievements in The Americas, Europe, and Asia-Pacific. This growth was widespread across all geographic regions and robust across all Services categories, as we set revenue records in cloud, payments, video, advertising, as well as records for the App Store and AppleCare in the December quarter. It’s important to note that last year included an extra week, making our underlying performance stronger than the reported 11%. The entire ecosystem is thriving, as reflected in the growth of both transacting accounts and paid accounts, both of which are critical metrics. Our paid subscriptions are continuing to grow at a strong double-digit rate, and we have surpassed one billion paid subscriptions across all service platforms, more than double what we had just four years ago, marking significant growth. Regarding the March quarter, I indicated during my prepared remarks that we expect to maintain a similar double-digit growth rate as we did for the December quarter. We do not provide specific guidance for the various service categories, but we will share more details when we report in three months.", 'sentiment': '0.4'}, {'speaker': 'Erik Woodring', 'title': 'Analyst', 'content': "Okay. Thank you, Luca. And then, Tim, really nice to see your installed base reach a record high, again, against all products and geos. I'm wondering if you could share a bit more detail about the new users you were able to onboard over the last 12 months. Meaning, how might this new cohort look different from past cohorts, either in terms of geographic representation or SKU to certain products or even how their monetization trends might differ from past cohorts. And that's it from me. Thanks so much.", 'sentiment': '0.2'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Yes. Hi, Erik. I would say emerging markets have been a very key area of strength for us. If you look at it, India grew in revenue terms strong double digits in the December quarter and hit a quarter revenue record. The other emerging markets like Indonesia also hit a quarterly record. And we had several regions, with records from Latin America to the Middle East. That theme has been pretty consistent across the other quarters of the year as well. And so, emerging markets are very, very important. And I feel like we are doing a great job there.', 'sentiment': '0.5'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thanks, Erik. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Mike Ng with Goldman Sachs. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Mike Ng', 'title': 'Analyst', 'content': "Good afternoon. I have two questions. First, regarding Services, I'm curious about the outlook for the March quarter. Are we expecting a similar double-digit growth rate as in the December quarter? I'm wondering why it might not increase, considering the December quarter faced challenges from an extra week of comparison. Also, I believe the pricing adjustments for some Apple One services made last winter should benefit the March quarter. It would be helpful to hear your assumptions on this. I also have a quick follow-up.", 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes. We'll see how the quarter develops. I would point to two things. One is the fact that we mentioned that we expect a couple of points of negative foreign exchange in the March quarter, and foreign exchange was essentially flat for us in the December quarter. So, you've got a bit of a headwind there. And then, when you look at our other progression of our Services business over the last few quarters, the compares for March are slightly more difficult than the compares for December.", 'sentiment': '0.2'}, {'speaker': 'Mike Ng', 'title': 'Analyst', 'content': 'Great. Thank you, Luca. And then, my second question. It was very interesting to hear about some of the enterprise customer investments into Vision Pro. Could you maybe just talk about some of the efforts to support the Vision Pro developer ecosystem? And it was also good to hear about the potential upcoming announcements on AI. So any thoughts there would also be helpful. Thank you.', 'sentiment': '0.2'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "We are incredibly excited about the Enterprise opportunities with Vision Pro. I've seen several demos from different companies, including Walmart's impressive merchandising app. There are also firms working on design collaboration apps and various field service applications. The applications vary widely, including those for control and command centers. SAP is strongly supporting this, which is significant since they work with so many companies. We see a great opportunity for us in Enterprise, and we couldn't be more excited about the current situation. We are looking forward to tomorrow, as this has been the result of years of effort from many people at Apple. It truly required a company-wide effort to reach this point. Regarding generative AI, we have a lot going on internally. Our approach has always been to first do the work and then discuss it, rather than getting ahead of ourselves. So, we will adhere to that strategy as well. However, we are working on some exciting developments that we will share later this year.", 'sentiment': '0.6'}, {'speaker': 'Mike Ng', 'title': 'Analyst', 'content': 'Wonderful. Thank you, Tim.', 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Yes.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Mike. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Wamsi Mohan with Bank of America. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Wamsi Mohan', 'title': 'Analyst', 'content': "Yes, thank you so much. I have two questions as well. First on iPhone. There have been concerns around replacement cycles lengthening, China competition intensifying, and you still beat iPhone revenues despite the weaker performance in China. Curious how you're thinking about the 15 cycle overall, given what you saw in the December quarter. And I have a follow-up.", 'sentiment': '0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Hi, it's Tim. We were up 6%, as we mentioned earlier, and we're pleased with that performance. We saw strong results in several regions, setting all-time records in Europe and the Asia-Pacific. We performed particularly well in emerging markets, including Latin America and the Middle East, and set records for the December quarter in India and Indonesia. There has been a fantastic response to the iPhone, with an all-time high for iPhone upgraders and the installed base reaching a new record. Additionally, iPhones accounted for four of the top five smartphone models in the U.S. and Japan, four of the top six in urban China and the UK, and all five in Australia. Customer satisfaction for the iPhone 15 reached 99%. Since its announcement and shipment in September, iPhone 15 has been outselling iPhone 14, and the record number of upgraders is particularly exciting for us.", 'sentiment': '0.6'}, {'speaker': 'Wamsi Mohan', 'title': 'Analyst', 'content': "Great. Thank you, Tim. And as a follow-up, obviously, you're just launching the Vision Pro and it's an entirely new category. It's a price point that's a much higher starting price point relative to most of your other, probably over the last decade, product introductions, but just wondering how would you measure the success of Vision Pro over time and which Apple products adoption curve would you look at as potentially the most similar? And is there a way in which we could think Vision Pro could eclipse maybe something like the iPad in revenue over time. Thank you.", 'sentiment': '0.5'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "You know, each product has its own journey. And so, I wouldn't want to compare it to any one in particular. I would just say we couldn't be more excited. Internally, we've got an incredible amount of excitement from developers and from customers that can't wait till tomorrow to pick up their units. And we are incredibly proud to be able to demo the unit in so many of our stores in the U.S. starting tomorrow for people that want to check it out. And so, we'll see and report the results of it in the Wearables category that you're familiar with. I think that if you look at it from a price point of view, there's an incredible amount of technology that's packed into the product. There's 5,000 patents in the product and it's, of course, built on many innovations that Apple has spent multiple years on, from silicon to displays and significant AI and machine learning. All the hand tracking, the room mapping, all of this stuff is driven by AI. And so, we're incredibly excited about it. I can't wait to be in the store tomorrow and see the reaction myself.", 'sentiment': '0.8'}, {'speaker': 'Wamsi Mohan', 'title': 'Analyst', 'content': 'Thank you so much, Tim.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': "Thanks, Wamsi. Operator, we'll take the next question please.", 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Amit Daryanani with Evercore. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Amit Daryanani', 'title': 'Analyst', 'content': "Good afternoon. I have two as well. I guess, first off, I was hoping you could talk a little bit about what you're seeing in China right now. I think from a geographic basis, one of the few places that was down double digits, while everything else was growing. So, I'm hoping you spend a bit of time discussing what are you seeing there from a competitive perspective and more importantly, from a demand perspective in China?", 'sentiment': '-0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "In Mainland China, the iPhone has been a major point of interest, and when we examine it in constant currency from an operational standpoint, we experienced a mid-single digit decline. Other factors contributed to a more significant year-over-year contraction. On the positive side, we saw strong growth in upgraders in Mainland China compared to the previous year, and we had four of the top six smartphone models in urban China. Additionally, IDC recently noted that we were the leading brand for both the entire year and the December quarter. While there are positive developments, we'd obviously prefer not to see a contraction.", 'sentiment': '0.1'}, {'speaker': 'Amit Daryanani', 'title': 'Analyst', 'content': 'Fair enough. And then, as a follow-up, you folks have implemented a fair bit of changes around the App Store in Europe post the DMA implementation there. Can you just touch on what are some of the key updates? And then, Luca, as a net of it all, do you see it having any significant impact financially to your services or a broader Apple P&L statement? Thank you.', 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Yes. Let me address both aspects, and then Luca can provide additional insights. Last week, we announced several changes in Europe set to take effect in March, which is the last month of the first calendar quarter and the second fiscal quarter. Among the changes are alternate billing options, different app stores, and our marketplaces. We are also enabling NFC for new banking and wallet app capabilities. These are some of the measures we've introduced. Over the years, we have focused on privacy, security, and usability. We have made every effort to retain the features that users love about our ecosystem, but we will inevitably be limited in what we can offer due to regulatory compliance. Predicting the choices that developers and users will make is quite challenging, so we will observe how things unfold in March.", 'sentiment': '0.0'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes, Amit. As Tim said, these are changes that we're going to be implementing in March. A lot will depend on the choices that will be made. Just to keep it in context, the changes applied to the EU market, which represents roughly 7% of our global app store revenue.", 'sentiment': '0.0'}, {'speaker': 'Amit Daryanani', 'title': 'Analyst', 'content': "Perfect. That's a really good perspective to have. Thank you very much.", 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Amit. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Aaron Rakers with Wells Fargo. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Aaron Rakers', 'title': 'Analyst', 'content': "Yes. Thanks for taking the question. I have two as well as you would imagine. I guess, the first question I wanted to just ask maybe unpack a little bit more, just remarkable trends that we're seeing in your product gross margin specifically. So, I'm curious as we look forward, I guess on this last quarter, where there any kind of benefits you're seeing from like just the purchase component, obligations that you've put in place, let's say, a year ago, and that flowing through. And how are you thinking about the component pricing environment as we think about that gross margin into the March quarter and looking-forward?", 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes. On the product side and then maybe I'll make a comment in total for the company. On the product side, our gross margins increased sequentially 280 basis points. So, obviously, a very significant increase. I would say the two primary components of the increase are a favorable mix. Of course, iPhone did very well. We did very well with our high-end models. And leverage, of course, it's the biggest quarter of the year for us, so we get the leverage effect. We had a partial offset, negative impact from foreign exchange. But net-net, obviously, very significant improvement. And we had very similar dynamics on the Services side where we increased sequentially 190 basis points, also, in this case, due to a more favorable mix. And so, the combined effect of the two businesses gave us the 45.9% at the total company level, which is up 70% sequentially. You've heard from my prepared remarks that we are guiding total company gross margin to 46% to 47%, which is an additional expansion of margins compared to the already very strong results of the December quarter.", 'sentiment': '0.4'}, {'speaker': 'Aaron Rakers', 'title': 'Analyst', 'content': "Okay. And then, the second question I was just going to ask. Tim, you alluded to kind of your excitement around generative AI and some announcements that we should think about maybe later this year. One of the things that stands out for me is that your capital expenditures have actually come down this last year. I'm curious as you look to lean in more to generative AI, is there something we should consider about the CapEx intensity at Apple to make investments to really set the table for generative AI, kind of platform as we move forward? Just given some of the things that we've seen from some other large tech companies.", 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "I'll take the question, Aaron. We've always said, we will never underinvest in the business. So, we are making all the investments that are necessary throughout our product development, software development, services development. And so, we will continue to invest in every area of the business and at the appropriate level. And we're very excited about what's in store for us for the rest of the year.", 'sentiment': '0.5'}, {'speaker': 'Aaron Rakers', 'title': 'Analyst', 'content': 'Thank you.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thanks, Aaron. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Krish Sankar with TD Cowen. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Krish Sankar', 'title': 'Analyst', 'content': "Yes, thank you for taking my question. I have two inquiries. First, for Luca, can you clarify whether the $5 billion impact in the March quarter pertains to product revenue or total company revenue? Additionally, you mentioned strong gross margins. While I understand some commodity costs were deflationary last year, it appears they are becoming inflationary now. Also, considering the Mac silicon conversions you've made, I'm trying to determine how much more potential there is to improve gross margins. I will follow up with Tim afterward.", 'sentiment': '0.0'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes. The first part of the question was about the $5 billion. A year ago, we encountered supply disruptions for the iPhone 14 Pro and Pro Max due to factory shutdowns caused by COVID-19. Consequently, there was pent-up demand that was fulfilled as we moved out of the December quarter, and we also addressed channel fill during the March quarter. The nearly $5 billion I mentioned is entirely related to the iPhone. Regarding gross margins, we are currently at very high levels and I've reiterated that we have experienced good expansion over the past few quarters. We are now projecting gross margins of 46% to 47%, which considers the commodity environment, foreign exchange conditions, and our product and services mix. The result of this is strong guidance, which we are pleased with.", 'sentiment': '0.4'}, {'speaker': 'Krish Sankar', 'title': 'Analyst', 'content': "Thanks a lot, Luca. And then I have a follow-up for Tim. Tim, it was very interesting to hear your comments on enterprise. And historically, Apple has been a consumer-centric company. And now with Vision Pro, Mac, it's sort of penetrating more into the enterprise. I'm kind of curious how to think about Apple of the future? Would it still be consumer-centric or do you think it's going to be more enterprise-focused also as we get into the future? Thank you very much.", 'sentiment': '0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "We've really concluded that we can do both. That if you look at it, what has happened over the last several years is that employees are in a position in many companies to choose their own technology that is the best for them. And so, it sort of took some of the central command from the traditional company and decentralized the decision-making. That is a huge advantage for Apple, because there's a lot of people out there that want to use a Mac. They're using a Mac at home. They'd like to use one in the office as well. iPad has also benefited from that. Vision Pro, when you look at the ton of use cases, I mean, we're starting with a million apps and 600 plus that have been designed particularly for Vision Pro. When I look at what is coming out of Enterprise, it's some of the most innovative things I've seen come out of Enterprise in a long time. And so, I think there's a potential fit for the Mac and iPad, and of course, iPhone has been in enterprise since the early days of the iPhone. I think there's a nice opportunity there for Vision Pro as well.", 'sentiment': '0.6'}, {'speaker': 'Krish Sankar', 'title': 'Analyst', 'content': 'Great. Thanks a lot Tim. Very interesting to hear. Thank you.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Krish. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from David Vogt with UBS. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'David Vogt', 'title': 'Analyst', 'content': "Thank you. I have two questions. Tim and Luca, I appreciate the strength in emerging markets like India and others mentioned on the call, but could you elaborate on the Americas? It seemed relatively flat. You discussed China, but what are you observing in that market with the carriers in the States? Is the sales cycle or the replacement cycle extending? What do you believe needs to change to potentially re-accelerate business in the Americas, especially in the iPhone segment? Also, I want to clarify the guidance. Considering the $5 billion pull forward last year in the March quarter from a channel fill perspective, even if I account for it this quarter or back it out from the last, the March quarter appears to be the softest since the COVID pandemic. I understand the Americas is a bit softer than China, as you clarified, but how do you perceive the macro conditions by region? Do you think we're approaching a trough in macro demand, or how long do you anticipate this weakness will last? Thank you.", 'sentiment': '-0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Let me address the first part of your question about the U.S. market. The U.S. continues to be the primary driver of our revenue, and we saw growth in the iPhone segment during the December quarter, with the install base reaching an all-time high. Measuring the replacement cycle is quite challenging at any given time. Internally, we concentrate on the active install base and sales over typical cycles, and we have a positive outlook on these metrics. In terms of market performance, the iPhone accounts for four out of the top five best-selling smartphones in the U.S., and customer satisfaction is remarkably high at 99%. We are very confident in our position in the U.S. market.', 'sentiment': '0.5'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "And I would add to that, keep in mind, obviously, the extra week that we had a year ago that obviously makes the compare a bit distorted. On the March quarter guide, I would point to you that, obviously, the COVID years had a lot of, let's say, turmoil in it, a lot of volatility that typically you wouldn't see. If you look at our sequential progression from December to March this year versus pre-COVID versus like a more normal environment, it's actually stronger than those years.", 'sentiment': '0.1'}, {'speaker': 'David Vogt', 'title': 'Analyst', 'content': 'Got it. Thanks, guys.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, David. Operator, can we have the last question, please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Thank you. Our last question is from Ben Reitzes with Melius Research. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Ben Reitzes', 'title': 'Analyst', 'content': 'Yes, hi. Thank you. I have two questions, if I may. First, I would like to clarify the situation in China. Tim, you mentioned last quarter that it was still perceived as a growth market. Given the concerns recently and what we observed in the last quarter, do you see any indicators that suggest the market could return to growth in the future? Are you still optimistic about this outlook? Secondly, I have a quick follow-up.', 'sentiment': '-0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Ben, we've been in China for 30 years. And I remain very optimistic about China over the long term. And I feel good about hitting a new install base number, high watermark and very good about the growth in upgraders year-over-year during the quarter.", 'sentiment': '0.5'}, {'speaker': 'Ben Reitzes', 'title': 'Analyst', 'content': "Great. Thanks, Tim. Regarding AI, while I understand you won't discuss your plans, do you believe that processing on smartphones and devices like yours will play a significant role in AI and AI applications, and that it's an opportunity you can capitalize on?", 'sentiment': '0.2'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Let me just say that I think there's a huge opportunity for Apple with GenAI and AI. And without getting into more details and getting out in front of myself.", 'sentiment': '0.5'}, {'speaker': 'Ben Reitzes', 'title': 'Analyst', 'content': 'Thanks Tim.', 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Yes, thanks, Ben.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': "All right. Thank you Ben. A replay of today's call will be available for two weeks on Apple Podcasts, as a webcast on apple.com/investor and via telephone. The number for the telephone replay is 866-583-1035. Please enter confirmation code 0106234 followed by the pound sign. These replays will be available by approximately 5 p.m. Pacific time today. Members of the press with additional questions can contact Josh Rosenstock at 408-862-1142 and financial analysts can contact me, Suhasini Chandramouli, with additional questions at 408-974-3123. Thanks again for joining us.", 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': "Once again, this does conclude today's conference. We do appreciate your participation.", 'sentiment': '0.0'}]}

In [104]:
b = a.get('transcript')

c = []

for x in b:
    c.append({
        'speaker': x['speaker'],
        'title': x['title'],
        'content': x['content']
    })

c

[{'speaker': 'Operator',
  'title': 'Operator',
  'content': "Good day, and welcome to the Apple Q1 Fiscal Year 2024 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions. I would like to turn the call over to Suhasini Chandramouli, Director of Investor Relations. Please go ahead."},
 {'speaker': 'Suhasini Chandramouli',
  'title': 'Director of Investor Relations',
  'content': "Thank you for joining us. Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Before turning the call over to Tim, I would like to remind everyone that the quarter we're reporting today included 13 weeks, whereas the quarter we reported a year ago included 14 weeks. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, o

In [110]:
c

response = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": f""" 
           You will be given the full earnings call transcript in {c}.

            Your task is to extract **7–10 investor-relevant insights** strictly based on what 
            **Apple management said** in the transcript.
            
            Rules you MUST follow:
            1. Use ONLY information explicitly stated by Apple executives (CEO, CFO, or Investor Relations).  
               Do NOT include analyst opinions, assumptions, or external interpretation.
            2. Each point must represent a **distinct insight**, not a restatement of another point.
            3. For every point, clearly mention **who said it** (name + role).
            4. Do NOT use generic phrases such as “strong performance”, “positive outlook”, or “solid results” 
            unless backed by specific data or commentary.
            5. Each point must explain **why this matters to a long-term investor** 
            (growth, margins, cash flow, competitive position, or strategic direction).
            6. Prefer **synthesis over repetition** — combine numbers with management commentary where relevant.
            7. Do NOT introduce any facts, estimates, or forward views not explicitly stated in the transcript.
            
            Output format:
            - Numbered list (7–10 points)
            - 2–4 sentences per point
            - End each point with: *(Source: Name, Title)*
            
            Tone:
            - Analytical
            - Investor-oriented
            - Neutral and factual

            Don't only include positive or negative points, try to include both
            """}
        ], temperature = 0.3
    )

print(response.choices[0].message.content)

1. Apple reported revenue of $119.6 billion for the December quarter, which is a 2% increase year-over-year despite having one less week in the quarter. This growth demonstrates Apple's resilience and ability to generate revenue even under challenging conditions, which is crucial for long-term investors focused on consistent revenue streams. *(Source: Tim Cook, CEO)*

2. The Services segment achieved an all-time revenue record of $23.1 billion, reflecting an 11% year-over-year increase. This growth is significant as it indicates a strong expansion of Apple's ecosystem, which is essential for driving recurring revenue and enhancing overall profitability. *(Source: Luca Maestri, CFO)*

3. Apple’s installed base of active devices surpassed 2.2 billion, marking a new record. This growing installed base is vital for long-term investors as it not only enhances customer loyalty but also provides a larger audience for Apple's services and products, facilitating future revenue growth. *(Source:

In [143]:
a = {'symbol': 'AAPL', 'quarter': '2024Q1', 'transcript': [{'speaker': 'Operator', 'title': 'Operator', 'content': "Good day, and welcome to the Apple Q1 Fiscal Year 2024 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions. I would like to turn the call over to Suhasini Chandramouli, Director of Investor Relations. Please go ahead.", 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': "Thank you for joining us. Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Before turning the call over to Tim, I would like to remind everyone that the quarter we're reporting today included 13 weeks, whereas the quarter we reported a year ago included 14 weeks. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding revenue, gross margin, operating expenses, other income and expense, taxes, capital allocation and future business outlook, including the potential impact of macroeconomic conditions on the company's business and results of operations. These statements involve risks and uncertainties that may cause actual results or trends to differ materially from our forecast. For more information, please refer to the risk factors discussed in Apple's most recently filed annual report on Form 10-K and the form 8-K filed with the SEC today along with the associated press release. Apple assumes no obligation to update any forward-looking statements, which speak only as of the date they are made. I'd now like to turn the call over to Tim for introductory remarks.", 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Thank you, Suhasini. Good afternoon, everyone, and thanks for joining the call. Today, Apple is reporting revenue of $119.6 billion for the December quarter, up 2% from a year ago despite having one less week in the quarter. EPS was $2.18, up 16% from a year ago and an all-time record. We achieved revenue records across more than two dozen countries and regions including all-time records in Europe and the rest of Asia-Pacific. We also continue to see strong double-digit growth in many emerging markets with all-time records in Malaysia, Mexico, The Philippines, Poland, and Turkey, as well as December quarter records in India, Indonesia, Saudi Arabia, and Chile. In Services, we set an all-time revenue record with paid subscriptions growing double digits year-over-year. And I'm pleased to announce today that we have set a new record for our installed base, which has now surpassed 2.2 billion active devices. We are announcing these results on the eve of what is sure to be a historic day as we enter the era of spatial computing. Starting tomorrow, Apple Vision Pro, the most advanced personal electronics device ever, will be available in Apple stores for customers in the U.S. with expansion to other countries later this year. Apple Vision Pro is a revolutionary device built on decades of Apple innovation and it's years ahead of anything else. Apple Vision Pro has a groundbreaking new input system and thousands of innovations, and it will unlock incredible experiences for users and developers that are simply not possible on any other device. There is already so much excitement behind this product from reviewers, customers, and developers. They are praising everything from the incredible experience of watching a movie on a 100-foot screen to remarkable new machine learning capabilities like hand tracking and room mapping. We can't wait for people to experience the magic for themselves. Moments like these are what we live for at Apple. They're why we do what we do. They're why we're so unflinchingly dedicated to groundbreaking innovation and why we're so focused on pushing technology to its limits as we work to enrich the lives of our users. As we look ahead, we will continue to invest in these and other technologies that will shape the future. That includes artificial intelligence, where we continue to spend a tremendous amount of time and effort, and we're excited to share the details of our ongoing work in that space later this year. Now, let's turn to the results for the December quarter, beginning with iPhone. We are proud to report that revenue came in at $69.7 billion, 6% higher than a year ago. The iPhone 15 lineup has earned glowing reviews and been embraced by customers. The iPhone 15 and iPhone 15 Plus feature a gorgeous new design with color-infused back glass and contoured edges, Dynamic Island, A16 Bionic, and a new 48 megapixel camera system. And the iPhone 15 Pro and iPhone 15 Pro Max set the gold standard for smartphones with a beautiful and lighter titanium design, industry-leading performance with A17 Pro, and our most advanced camera system with the equivalent of seven pro lenses and the ability to record spatial video. Features like Emergency SOS and roadside assistance via satellite bring peace of mind to users when they travel, and I'm grateful for every note I've received about their lifesaving impact. Turning to Mac. Revenue came in at $7.8 billion, up 1% year-over-year, driven by the strength of our latest M3-powered MacBook Pro models in spite of having one less week of sales. Just last week, we got to wish Mac a happy 40th birthday. When it was introduced 40 years ago, Mac changed everything, and through the years, it has done so again and again. Recently, we have been on a tremendous pace of innovation. Since the introduction of Apple silicon in 2020, we've been proud to offer our users unmatched performance and power along with a remarkable Neural Engine for artificial intelligence and machine learning. This past fall, we had an amazing launch of the latest generation of Apple silicon for Mac, M3, M3 Pro, and M3 Max. These chips break new ground in power and performance empowering users to do more than they ever could before, whether they're making a musical masterpiece using the latest features in Logic Pro, or beating their high score in a graphics intensive game. A favorite amongst students, business owners, artists, and video editors, our MacBook Pro lineup is the world's best pro notebook family. And iMac, the world's most capable and best-selling all-in-one, is now faster than ever, thanks to M3. In iPad, revenue for the December quarter was $7 billion, down 25% year-over-year due to a difficult comparison with the launch of the M2 iPad Pro and the 10th generation iPad during the December quarter last year and one less week of sales. iPad remains the most versatile, capable, and elegant tablet on the market today. It continues to be the go-to device for students, creators, and more with customers loving iPad's incredible combination of portability and performance. Powerful apps like Final Cut Pro and Logic Pro for iPad allow video and music creators to unleash their creativity in new ways that are only possible on iPad. iPad continues to push the boundaries of what's possible on a tablet. In Wearables, Home, and Accessories, revenue came in at $12 billion, down 11% from a year ago due to a difficult comparison with the launch timing of several products in this category and the impact of the 14th week last year. Across our latest Apple Watch lineup, we're enabling and encouraging our users to live a healthier day, while making Apple Watch even more intuitive to use. The new double-tap gesture on Apple Watch Series 9 and Apple Watch Ultra 2 makes it easier to answer calls, play and pause music, or take a photo with iPhone. I've been deeply moved by the many touching stories about how features like a regular rhythm notification and fall detection helped Apple Watch users when they needed it most. And for the first time ever, users can choose a carbon-neutral option of any new Apple Watch. Meanwhile, our AirPods lineup continues to be a holiday favorite. In Services, we set an all-time revenue record of $23.1 billion, an 11% year-over-year increase. Because we had one less week this quarter, this growth represents an acceleration from the September quarter, and we achieved all-time revenue records across advertising, cloud services, payment services, and video, as well as December quarter records in the App Store and AppleCare. Across our services, we're constantly growing our offerings to give users even more to love. With the redesigned Apple TV app, we've made it easier for subscribers to enjoy all their favorite shows, movies, and sports, including Apple TV+ hits like Masters of the Air, Monarch, and Slow Horses. We're proud to be a part of Martin Scorsese's Killers of the Flower Moon, a film that has moved audiences and earned more than 200 accolades including Best Film of the Year from the New York Film Critics Circle, nine BAFTA nominations, a Golden Globe win, and 10 Oscar nominations, including Best Picture. Across all Apple TV+ productions, we've now earned 2050 award nominations and 450 wins since we've introduced the service. We're also excited to have a new season of Major League Soccer kicking off this month. We're looking forward to seeing Lionel Messi return to the field and to following all of our favorite teams in what is sure to be an incredible season. And we're counting down to the Apple Music Super Bowl halftime show, featuring Usher. Turning to Retail. In recent months, we opened three stores, including our 100th store in Asia-Pacific. Throughout the holidays, our team members pulled out all the stops to help customers find the perfect gift. And I know our U.S. team members are especially excited to begin demoing Apple Vision Pro for our customers tomorrow. At Apple, we live and breathe innovation. We are driven to pioneer new technology that can enrich our customers' lives, and we're just as intentional about showing up with our values and being a force for good in the world. February is Black History Month, and to honor it, we've launched our new Black Unity Collection, which includes the Black Unity Sport Loop band. This year's designs reflect a lasting commitment to working toward a more equitable world. We also continue to do key work through our Racial Equity and Justice Initiative, and we're proud to continue providing grants to organizations that are making a real impact in the world. In recent months, we've also taken significant strides in our environmental work. We're partnering with suppliers to bring more clean energy online for Apple production. We're using more recycled materials than ever before and more energy-efficient transportation than ever before. And each day, we are taking more and more steps toward becoming 100% carbon-neutral across all of our products by 2030. Apple is a company that has never shied away from big challenges. That's because we are grounded by a deep sense of purpose and guided by a core belief in the transformative power of innovation. And so, we are optimistic about the future, confident in the long term, and as excited as we've ever been to deliver for our users like only Apple can. With that, I'll turn it over to Luca.", 'sentiment': '0.7'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Thank you, Tim, and good afternoon, everyone. Revenue for the December quarter was $119.6 billion, up 2% from last year. During the December quarter a year ago, two unique factors affected our results. First, we had an additional week in the quarter. And second, we had COVID-related factory shutdowns that limited iPhone supply. We estimate that the net impact of these two factors resulted in a 2 percentage point headwind to our revenue performance this quarter. We set all-time revenue records in Europe and the rest of Asia-Pacific and continue to see strong performance across our emerging markets with double-digit growth in the majority of the emerging markets we track. Products revenue was $96.5 billion, flat compared to last year, driven by strength in iPhone, offset by challenging comparisons for iPad and Wearables, Home, and Accessories and one less week of sales this year across the entire portfolio. Thanks to our unparalleled customer loyalty and very strong levels of customer satisfaction, our total installed base of active devices set a new record across all products and all geographic segments, and is now over 2.2 billion active devices. Services revenue set an all-time record of $23.1 billion, up 11% year-over-year. When we take into account the extra week last year, this represents a sequential acceleration of growth from the September quarter. We are very pleased with our Services performance in both developed and emerging markets with all-time revenue records in the Americas, Europe, and the rest of Asia-Pacific. Company gross margin was 45.9%, up 70 basis points sequentially, driven by leverage and favorable mix, partially offset by foreign exchange. Products gross margin was 39.4%, up 280 basis points sequentially, also driven by leverage and mix, partially offset by foreign exchange. Services gross margin was 72.8%, up 190 basis points from last quarter, due to a more favorable mix. Operating expenses of $14.5 billion were at the midpoint of the guidance range we provided and up 1% year-over-year. Net income was $33.9 billion, up $3.9 billion from last year. Diluted EPS was $2.18, up 16% versus last year and an all-time record. And operating cash flow was very strong at $39.9 billion. Let me now provide more detail for each of our revenue categories. iPhone revenue was $69.7 billion, up 6% year-over-year. We set all-time records in several countries and regions, including Latin America, Western Europe, the Middle East, and Korea, as well as December quarter records in India and Indonesia. Our iPhone active installed base grew to a new all-time high, and we had an all-time record number of iPhone upgraders during the quarter. Customers are loving their new iPhone 15 family, with the latest reports from 451 Research indicating customer satisfaction of 99% in the U.S. In fact, many iPhone models were among the top-selling smartphones around the world during the quarter. According to a survey from Kantar, iPhones were four out of the top five models in the U.S. and Japan, four out of the top six models in urban China and the UK, and all top five models in Australia. Mac generated revenue of $7.8 billion and returned to growth, despite one less week of sales this year. This represents a significant acceleration from the September quarter when we faced a challenging compare due to the supply disruptions and subsequent demand recapture we experienced a year ago. Customer response to our latest iMac and MacBook Pro models powered by the M3 chips has been great. And our Mac installed base reached an all-time high with almost half of Mac buyers during the quarter being new to the product. Also, 451 Research recently reported customer satisfaction of 97% for Mac in the U.S. iPad generated $7 billion in revenue, down 25% year-over-year. iPad faced a difficult compare because during the December quarter last year, we launched the new iPad Pro and iPad 10 generation, and we had an extra week of sales. However, the iPad installed base continues to grow and is at an all-time high with over half of the customers who purchased iPads during the quarter being new to the product, and customer satisfaction for iPad was recently measured at 98% in the U.S. Wearables, Home, and Accessories revenue was $12 billion, down 11% year-over-year due to a challenging launch comparison and the extra week a year ago. This time last year, we had the full quarter benefit from the launches of the AirPods Pro 2nd generation, the Watch SE, and the first Watch Ultra. We continue to attract new customers to Apple Watch. Nearly two-thirds of customers purchasing an Apple Watch during the quarter were new to the product, and the latest reports from 451 Research indicate customer satisfaction of 96% in the U.S. And in Services, we were very pleased with our double-digit growth, which was driven by the strength of our ecosystem. Our installed base is now over 2.2 billion active devices and continues to grow nicely, establishing a solid foundation for the future expansion of our Services business. And we continue to see increased customer engagement with our services. Both transacting accounts and paid accounts reached a new all-time high, with paid accounts growing double digits year-over-year. Also, our paid subscriptions showed strong double-digit growth. We have well over 1 billion paid subscriptions across the services on our platform, more than double the number that we had only four years ago. Finally, we continue to build on the breadth and quality of our current services. From Oscar-nominated theatrical releases with Apple TV+ to more publications or News+ like The Atlantic and exciting new games on Arcade. Turning to Enterprise, we continue to see many business customers leverage Apple products to improve productivity and drive innovation. Target recently added the latest M3 MacBook Pro to their existing deployment of thousands of Macs, enabling employees across various departments to do their best work. In emerging markets, Zoho, a leading technology company headquartered in India, offers its 15,000-plus global employees a choice of devices, with 80% of their workforce using iPhone for work and nearly two-thirds of them choosing Mac as their primary computer. With the upcoming launch of Apple Vision Pro, we are seeing strong excitement in Enterprise. Leading organizations across many industries such as Walmart, Nike, Vanguard, Stryker, Bloomberg, and SAP have started leveraging and investing in Apple Vision Pro as their new platform to bring innovative spatial computing experiences to their customers and employees. From everyday productivity to collaborative product design to immersive training, we cannot wait to see the amazing things our enterprise customers will create in the months and years to come. Let me now turn to our cash position and capital return program. We ended the quarter with $173 billion in cash and marketable securities. We decreased commercial paper by $4 billion, leaving us with total debt of $108 billion. As a result, net cash was $65 billion at the end of the quarter, and our goal of becoming net cash-neutral over time remains unchanged. During the quarter, we returned nearly $27 billion to shareholders, including $3.8 billion in dividends and equivalents and $20.5 billion through open market repurchases of 112 million Apple shares. We also retired an additional 6 million shares in the final settlement of our 19th ASR. As usual, we will provide an update to our capital return program when we report results at the end of this quarter. As we move ahead into the March quarter, I'd like to review our outlook, which includes the types of forward-looking information that Suhasini referred to at the beginning of the call. The color we are providing today assumes that the macroeconomic outlook doesn't worsen from what we are projecting today for the current quarter. And we expect foreign exchange to be a revenue headwind of about 2 percentage points on a year-over-year basis. As a reminder, in the December quarter a year ago, we faced significant supply constraints on the iPhone 14 Pro and 14 Pro Max due to COVID-19 factory shutdowns. And in the March quarter a year ago, we were able to replenish channel inventory and fulfill significant pent-up demand from the constraints. We estimate that this impact added close to $5 billion to the March quarter's total revenue last year. When we remove this impact from last year's revenue, we expect both our March quarter total company revenue and iPhone revenue to be similar to a year ago. For our Services business, we expect a similar double-digit growth rate to what we reported in the December quarter. We expect gross margin to be between 46% and 47%. We expect OpEx to be between $14.3 billion and $14.5 billion. We expect OI&E to be around $50 million, excluding any potential impact from the mark-to-market of minority investments and our tax rate to be around 16%. Finally, today our Board of Directors has declared a cash dividend of $0.24 per share of common stock payable on February 15, 2024, to shareholders of record as of February 12, 2024. With that, let's open the call to questions.", 'sentiment': '0.5'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Luca. We ask that you limit yourself to two questions. Operator, may we have the first question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': "Certainly. We'll go ahead and take our first question from Erik Woodring with Morgan Stanley. Please go ahead.", 'sentiment': '0.0'}, {'speaker': 'Erik Woodring', 'title': 'Analyst', 'content': 'Hey guys, good evening. Thank you for taking my questions. I have two. And congrats on the nice quarter here. Luca, maybe if we start with you. Can you unpackage some of the Services drivers a bit for us? Obviously, really nice outperformance in the December quarter versus your expectations. A record gross margin implies your higher margin businesses were likely the sources of outperformance. But can you maybe just clarify a bit how we should think about Services growth for the March quarter? And then, speak to some of those underlying drivers in the December quarter and then in the March quarter, what the different puts and takes would be? And then, I have a follow-up please. Thank you.', 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Thank you for the question. Let's begin with the December quarter. As mentioned, we saw an 11% increase, reaching $23.1 billion, which is a record for us, with significant achievements in The Americas, Europe, and Asia-Pacific. This growth was widespread across all geographic regions and robust across all Services categories, as we set revenue records in cloud, payments, video, advertising, as well as records for the App Store and AppleCare in the December quarter. It’s important to note that last year included an extra week, making our underlying performance stronger than the reported 11%. The entire ecosystem is thriving, as reflected in the growth of both transacting accounts and paid accounts, both of which are critical metrics. Our paid subscriptions are continuing to grow at a strong double-digit rate, and we have surpassed one billion paid subscriptions across all service platforms, more than double what we had just four years ago, marking significant growth. Regarding the March quarter, I indicated during my prepared remarks that we expect to maintain a similar double-digit growth rate as we did for the December quarter. We do not provide specific guidance for the various service categories, but we will share more details when we report in three months.", 'sentiment': '0.4'}, {'speaker': 'Erik Woodring', 'title': 'Analyst', 'content': "Okay. Thank you, Luca. And then, Tim, really nice to see your installed base reach a record high, again, against all products and geos. I'm wondering if you could share a bit more detail about the new users you were able to onboard over the last 12 months. Meaning, how might this new cohort look different from past cohorts, either in terms of geographic representation or SKU to certain products or even how their monetization trends might differ from past cohorts. And that's it from me. Thanks so much.", 'sentiment': '0.2'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Yes. Hi, Erik. I would say emerging markets have been a very key area of strength for us. If you look at it, India grew in revenue terms strong double digits in the December quarter and hit a quarter revenue record. The other emerging markets like Indonesia also hit a quarterly record. And we had several regions, with records from Latin America to the Middle East. That theme has been pretty consistent across the other quarters of the year as well. And so, emerging markets are very, very important. And I feel like we are doing a great job there.', 'sentiment': '0.5'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thanks, Erik. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Mike Ng with Goldman Sachs. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Mike Ng', 'title': 'Analyst', 'content': "Good afternoon. I have two questions. First, regarding Services, I'm curious about the outlook for the March quarter. Are we expecting a similar double-digit growth rate as in the December quarter? I'm wondering why it might not increase, considering the December quarter faced challenges from an extra week of comparison. Also, I believe the pricing adjustments for some Apple One services made last winter should benefit the March quarter. It would be helpful to hear your assumptions on this. I also have a quick follow-up.", 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes. We'll see how the quarter develops. I would point to two things. One is the fact that we mentioned that we expect a couple of points of negative foreign exchange in the March quarter, and foreign exchange was essentially flat for us in the December quarter. So, you've got a bit of a headwind there. And then, when you look at our other progression of our Services business over the last few quarters, the compares for March are slightly more difficult than the compares for December.", 'sentiment': '0.2'}, {'speaker': 'Mike Ng', 'title': 'Analyst', 'content': 'Great. Thank you, Luca. And then, my second question. It was very interesting to hear about some of the enterprise customer investments into Vision Pro. Could you maybe just talk about some of the efforts to support the Vision Pro developer ecosystem? And it was also good to hear about the potential upcoming announcements on AI. So any thoughts there would also be helpful. Thank you.', 'sentiment': '0.2'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "We are incredibly excited about the Enterprise opportunities with Vision Pro. I've seen several demos from different companies, including Walmart's impressive merchandising app. There are also firms working on design collaboration apps and various field service applications. The applications vary widely, including those for control and command centers. SAP is strongly supporting this, which is significant since they work with so many companies. We see a great opportunity for us in Enterprise, and we couldn't be more excited about the current situation. We are looking forward to tomorrow, as this has been the result of years of effort from many people at Apple. It truly required a company-wide effort to reach this point. Regarding generative AI, we have a lot going on internally. Our approach has always been to first do the work and then discuss it, rather than getting ahead of ourselves. So, we will adhere to that strategy as well. However, we are working on some exciting developments that we will share later this year.", 'sentiment': '0.6'}, {'speaker': 'Mike Ng', 'title': 'Analyst', 'content': 'Wonderful. Thank you, Tim.', 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Yes.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Mike. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Wamsi Mohan with Bank of America. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Wamsi Mohan', 'title': 'Analyst', 'content': "Yes, thank you so much. I have two questions as well. First on iPhone. There have been concerns around replacement cycles lengthening, China competition intensifying, and you still beat iPhone revenues despite the weaker performance in China. Curious how you're thinking about the 15 cycle overall, given what you saw in the December quarter. And I have a follow-up.", 'sentiment': '0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Hi, it's Tim. We were up 6%, as we mentioned earlier, and we're pleased with that performance. We saw strong results in several regions, setting all-time records in Europe and the Asia-Pacific. We performed particularly well in emerging markets, including Latin America and the Middle East, and set records for the December quarter in India and Indonesia. There has been a fantastic response to the iPhone, with an all-time high for iPhone upgraders and the installed base reaching a new record. Additionally, iPhones accounted for four of the top five smartphone models in the U.S. and Japan, four of the top six in urban China and the UK, and all five in Australia. Customer satisfaction for the iPhone 15 reached 99%. Since its announcement and shipment in September, iPhone 15 has been outselling iPhone 14, and the record number of upgraders is particularly exciting for us.", 'sentiment': '0.6'}, {'speaker': 'Wamsi Mohan', 'title': 'Analyst', 'content': "Great. Thank you, Tim. And as a follow-up, obviously, you're just launching the Vision Pro and it's an entirely new category. It's a price point that's a much higher starting price point relative to most of your other, probably over the last decade, product introductions, but just wondering how would you measure the success of Vision Pro over time and which Apple products adoption curve would you look at as potentially the most similar? And is there a way in which we could think Vision Pro could eclipse maybe something like the iPad in revenue over time. Thank you.", 'sentiment': '0.5'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "You know, each product has its own journey. And so, I wouldn't want to compare it to any one in particular. I would just say we couldn't be more excited. Internally, we've got an incredible amount of excitement from developers and from customers that can't wait till tomorrow to pick up their units. And we are incredibly proud to be able to demo the unit in so many of our stores in the U.S. starting tomorrow for people that want to check it out. And so, we'll see and report the results of it in the Wearables category that you're familiar with. I think that if you look at it from a price point of view, there's an incredible amount of technology that's packed into the product. There's 5,000 patents in the product and it's, of course, built on many innovations that Apple has spent multiple years on, from silicon to displays and significant AI and machine learning. All the hand tracking, the room mapping, all of this stuff is driven by AI. And so, we're incredibly excited about it. I can't wait to be in the store tomorrow and see the reaction myself.", 'sentiment': '0.8'}, {'speaker': 'Wamsi Mohan', 'title': 'Analyst', 'content': 'Thank you so much, Tim.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': "Thanks, Wamsi. Operator, we'll take the next question please.", 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Amit Daryanani with Evercore. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Amit Daryanani', 'title': 'Analyst', 'content': "Good afternoon. I have two as well. I guess, first off, I was hoping you could talk a little bit about what you're seeing in China right now. I think from a geographic basis, one of the few places that was down double digits, while everything else was growing. So, I'm hoping you spend a bit of time discussing what are you seeing there from a competitive perspective and more importantly, from a demand perspective in China?", 'sentiment': '-0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "In Mainland China, the iPhone has been a major point of interest, and when we examine it in constant currency from an operational standpoint, we experienced a mid-single digit decline. Other factors contributed to a more significant year-over-year contraction. On the positive side, we saw strong growth in upgraders in Mainland China compared to the previous year, and we had four of the top six smartphone models in urban China. Additionally, IDC recently noted that we were the leading brand for both the entire year and the December quarter. While there are positive developments, we'd obviously prefer not to see a contraction.", 'sentiment': '0.1'}, {'speaker': 'Amit Daryanani', 'title': 'Analyst', 'content': 'Fair enough. And then, as a follow-up, you folks have implemented a fair bit of changes around the App Store in Europe post the DMA implementation there. Can you just touch on what are some of the key updates? And then, Luca, as a net of it all, do you see it having any significant impact financially to your services or a broader Apple P&L statement? Thank you.', 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Yes. Let me address both aspects, and then Luca can provide additional insights. Last week, we announced several changes in Europe set to take effect in March, which is the last month of the first calendar quarter and the second fiscal quarter. Among the changes are alternate billing options, different app stores, and our marketplaces. We are also enabling NFC for new banking and wallet app capabilities. These are some of the measures we've introduced. Over the years, we have focused on privacy, security, and usability. We have made every effort to retain the features that users love about our ecosystem, but we will inevitably be limited in what we can offer due to regulatory compliance. Predicting the choices that developers and users will make is quite challenging, so we will observe how things unfold in March.", 'sentiment': '0.0'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes, Amit. As Tim said, these are changes that we're going to be implementing in March. A lot will depend on the choices that will be made. Just to keep it in context, the changes applied to the EU market, which represents roughly 7% of our global app store revenue.", 'sentiment': '0.0'}, {'speaker': 'Amit Daryanani', 'title': 'Analyst', 'content': "Perfect. That's a really good perspective to have. Thank you very much.", 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Amit. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Aaron Rakers with Wells Fargo. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Aaron Rakers', 'title': 'Analyst', 'content': "Yes. Thanks for taking the question. I have two as well as you would imagine. I guess, the first question I wanted to just ask maybe unpack a little bit more, just remarkable trends that we're seeing in your product gross margin specifically. So, I'm curious as we look forward, I guess on this last quarter, where there any kind of benefits you're seeing from like just the purchase component, obligations that you've put in place, let's say, a year ago, and that flowing through. And how are you thinking about the component pricing environment as we think about that gross margin into the March quarter and looking-forward?", 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes. On the product side and then maybe I'll make a comment in total for the company. On the product side, our gross margins increased sequentially 280 basis points. So, obviously, a very significant increase. I would say the two primary components of the increase are a favorable mix. Of course, iPhone did very well. We did very well with our high-end models. And leverage, of course, it's the biggest quarter of the year for us, so we get the leverage effect. We had a partial offset, negative impact from foreign exchange. But net-net, obviously, very significant improvement. And we had very similar dynamics on the Services side where we increased sequentially 190 basis points, also, in this case, due to a more favorable mix. And so, the combined effect of the two businesses gave us the 45.9% at the total company level, which is up 70% sequentially. You've heard from my prepared remarks that we are guiding total company gross margin to 46% to 47%, which is an additional expansion of margins compared to the already very strong results of the December quarter.", 'sentiment': '0.4'}, {'speaker': 'Aaron Rakers', 'title': 'Analyst', 'content': "Okay. And then, the second question I was just going to ask. Tim, you alluded to kind of your excitement around generative AI and some announcements that we should think about maybe later this year. One of the things that stands out for me is that your capital expenditures have actually come down this last year. I'm curious as you look to lean in more to generative AI, is there something we should consider about the CapEx intensity at Apple to make investments to really set the table for generative AI, kind of platform as we move forward? Just given some of the things that we've seen from some other large tech companies.", 'sentiment': '0.1'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "I'll take the question, Aaron. We've always said, we will never underinvest in the business. So, we are making all the investments that are necessary throughout our product development, software development, services development. And so, we will continue to invest in every area of the business and at the appropriate level. And we're very excited about what's in store for us for the rest of the year.", 'sentiment': '0.5'}, {'speaker': 'Aaron Rakers', 'title': 'Analyst', 'content': 'Thank you.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thanks, Aaron. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from Krish Sankar with TD Cowen. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Krish Sankar', 'title': 'Analyst', 'content': "Yes, thank you for taking my question. I have two inquiries. First, for Luca, can you clarify whether the $5 billion impact in the March quarter pertains to product revenue or total company revenue? Additionally, you mentioned strong gross margins. While I understand some commodity costs were deflationary last year, it appears they are becoming inflationary now. Also, considering the Mac silicon conversions you've made, I'm trying to determine how much more potential there is to improve gross margins. I will follow up with Tim afterward.", 'sentiment': '0.0'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "Yes. The first part of the question was about the $5 billion. A year ago, we encountered supply disruptions for the iPhone 14 Pro and Pro Max due to factory shutdowns caused by COVID-19. Consequently, there was pent-up demand that was fulfilled as we moved out of the December quarter, and we also addressed channel fill during the March quarter. The nearly $5 billion I mentioned is entirely related to the iPhone. Regarding gross margins, we are currently at very high levels and I've reiterated that we have experienced good expansion over the past few quarters. We are now projecting gross margins of 46% to 47%, which considers the commodity environment, foreign exchange conditions, and our product and services mix. The result of this is strong guidance, which we are pleased with.", 'sentiment': '0.4'}, {'speaker': 'Krish Sankar', 'title': 'Analyst', 'content': "Thanks a lot, Luca. And then I have a follow-up for Tim. Tim, it was very interesting to hear your comments on enterprise. And historically, Apple has been a consumer-centric company. And now with Vision Pro, Mac, it's sort of penetrating more into the enterprise. I'm kind of curious how to think about Apple of the future? Would it still be consumer-centric or do you think it's going to be more enterprise-focused also as we get into the future? Thank you very much.", 'sentiment': '0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "We've really concluded that we can do both. That if you look at it, what has happened over the last several years is that employees are in a position in many companies to choose their own technology that is the best for them. And so, it sort of took some of the central command from the traditional company and decentralized the decision-making. That is a huge advantage for Apple, because there's a lot of people out there that want to use a Mac. They're using a Mac at home. They'd like to use one in the office as well. iPad has also benefited from that. Vision Pro, when you look at the ton of use cases, I mean, we're starting with a million apps and 600 plus that have been designed particularly for Vision Pro. When I look at what is coming out of Enterprise, it's some of the most innovative things I've seen come out of Enterprise in a long time. And so, I think there's a potential fit for the Mac and iPad, and of course, iPhone has been in enterprise since the early days of the iPhone. I think there's a nice opportunity there for Vision Pro as well.", 'sentiment': '0.6'}, {'speaker': 'Krish Sankar', 'title': 'Analyst', 'content': 'Great. Thanks a lot Tim. Very interesting to hear. Thank you.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, Krish. Operator, can we have the next question please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Our next question is from David Vogt with UBS. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'David Vogt', 'title': 'Analyst', 'content': "Thank you. I have two questions. Tim and Luca, I appreciate the strength in emerging markets like India and others mentioned on the call, but could you elaborate on the Americas? It seemed relatively flat. You discussed China, but what are you observing in that market with the carriers in the States? Is the sales cycle or the replacement cycle extending? What do you believe needs to change to potentially re-accelerate business in the Americas, especially in the iPhone segment? Also, I want to clarify the guidance. Considering the $5 billion pull forward last year in the March quarter from a channel fill perspective, even if I account for it this quarter or back it out from the last, the March quarter appears to be the softest since the COVID pandemic. I understand the Americas is a bit softer than China, as you clarified, but how do you perceive the macro conditions by region? Do you think we're approaching a trough in macro demand, or how long do you anticipate this weakness will last? Thank you.", 'sentiment': '-0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Let me address the first part of your question about the U.S. market. The U.S. continues to be the primary driver of our revenue, and we saw growth in the iPhone segment during the December quarter, with the install base reaching an all-time high. Measuring the replacement cycle is quite challenging at any given time. Internally, we concentrate on the active install base and sales over typical cycles, and we have a positive outlook on these metrics. In terms of market performance, the iPhone accounts for four out of the top five best-selling smartphones in the U.S., and customer satisfaction is remarkably high at 99%. We are very confident in our position in the U.S. market.', 'sentiment': '0.5'}, {'speaker': 'Luca Maestri', 'title': 'CFO', 'content': "And I would add to that, keep in mind, obviously, the extra week that we had a year ago that obviously makes the compare a bit distorted. On the March quarter guide, I would point to you that, obviously, the COVID years had a lot of, let's say, turmoil in it, a lot of volatility that typically you wouldn't see. If you look at our sequential progression from December to March this year versus pre-COVID versus like a more normal environment, it's actually stronger than those years.", 'sentiment': '0.1'}, {'speaker': 'David Vogt', 'title': 'Analyst', 'content': 'Got it. Thanks, guys.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': 'Thank you, David. Operator, can we have the last question, please?', 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': 'Thank you. Our last question is from Ben Reitzes with Melius Research. Please go ahead.', 'sentiment': '0.0'}, {'speaker': 'Ben Reitzes', 'title': 'Analyst', 'content': 'Yes, hi. Thank you. I have two questions, if I may. First, I would like to clarify the situation in China. Tim, you mentioned last quarter that it was still perceived as a growth market. Given the concerns recently and what we observed in the last quarter, do you see any indicators that suggest the market could return to growth in the future? Are you still optimistic about this outlook? Secondly, I have a quick follow-up.', 'sentiment': '-0.1'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Ben, we've been in China for 30 years. And I remain very optimistic about China over the long term. And I feel good about hitting a new install base number, high watermark and very good about the growth in upgraders year-over-year during the quarter.", 'sentiment': '0.5'}, {'speaker': 'Ben Reitzes', 'title': 'Analyst', 'content': "Great. Thanks, Tim. Regarding AI, while I understand you won't discuss your plans, do you believe that processing on smartphones and devices like yours will play a significant role in AI and AI applications, and that it's an opportunity you can capitalize on?", 'sentiment': '0.2'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': "Let me just say that I think there's a huge opportunity for Apple with GenAI and AI. And without getting into more details and getting out in front of myself.", 'sentiment': '0.5'}, {'speaker': 'Ben Reitzes', 'title': 'Analyst', 'content': 'Thanks Tim.', 'sentiment': '0.0'}, {'speaker': 'Tim Cook', 'title': 'CEO', 'content': 'Yes, thanks, Ben.', 'sentiment': '0.0'}, {'speaker': 'Suhasini Chandramouli', 'title': 'Director of Investor Relations', 'content': "All right. Thank you Ben. A replay of today's call will be available for two weeks on Apple Podcasts, as a webcast on apple.com/investor and via telephone. The number for the telephone replay is 866-583-1035. Please enter confirmation code 0106234 followed by the pound sign. These replays will be available by approximately 5 p.m. Pacific time today. Members of the press with additional questions can contact Josh Rosenstock at 408-862-1142 and financial analysts can contact me, Suhasini Chandramouli, with additional questions at 408-974-3123. Thanks again for joining us.", 'sentiment': '0.0'}, {'speaker': 'Operator', 'title': 'Operator', 'content': "Once again, this does conclude today's conference. We do appreciate your participation.", 'sentiment': '0.0'}]}


a.get('transcript')

[{'speaker': 'Operator',
  'title': 'Operator',
  'content': "Good day, and welcome to the Apple Q1 Fiscal Year 2024 Earnings Conference Call. Today's call is being recorded. At this time, for opening remarks and introductions. I would like to turn the call over to Suhasini Chandramouli, Director of Investor Relations. Please go ahead.",
  'sentiment': '0.0'},
 {'speaker': 'Suhasini Chandramouli',
  'title': 'Director of Investor Relations',
  'content': "Thank you for joining us. Speaking first today is Apple's CEO, Tim Cook; and he'll be followed by CFO, Luca Maestri. After that, we'll open the call to questions from analysts. Before turning the call over to Tim, I would like to remind everyone that the quarter we're reporting today included 13 weeks, whereas the quarter we reported a year ago included 14 weeks. Please note that some of the information you'll hear during our discussion today will consist of forward-looking statements, including, without limitation, those regarding re